# Loading Packages

In [1]:
! jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

## Seaborn for plots

In [3]:
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')

## Progress Bar

In [4]:
%%capture
from tqdm import tqdm_notebook
tqdm_notebook().pandas()

## Pandas Options

In [5]:
pd.set_option('max_columns', None)
pd.set_option('display.max_rows', None)

# Loading Files

## 1 - Read Default Data

In [6]:
path = '..//resources//aaa_sample_data.xlsx'

df_original = pd.read_excel(path)

* Data Glance

In [7]:
df_original.head(n=2)

,Column1,Individual Key,Household Key,Member Flag,City,State - Grouped,ZIP5,ZIP9,FSV CMSI Flag,FSV Credit Card Flag,FSV Deposit Program Flag,FSV Home Equity Flag,FSV ID Theft Flag,FSV Mortgage Flag,INS Client Flag,TRV Globalware Flag,Number of Children,Responded to Catalog,Race,Length Of Residence,Mail Responder,Home Owner,Income,Date Of Birth,Children,Education,Dwelling Type,Credit Ranges,Language,Gender,Active Expiration Date,Address Change Date,Bad Address Flag,Billing Code Description,Birth Date MMDDYYYY,Branch Name,Cancel Date,Cancel Reason,County,Do Not Direct Mail Solicit,Email Available,Email Status,ERS ENT Count Year 1,ERS ENT Count Year 2,ERS ENT Count Year 3,ERS Member Cost Year 1,ERS Member Cost Year 2,ERS Member Cost Year 3,Right_Gender,Right_Individual Key,Join AAA Date,Join Club Date,Member Key,Member Map Location,Member Number Associate ID,Member Phone Type,Member Status,Member Tenure Years,Member Type,Membership ID,Months from Join to Cancel,Opt-Out - Publication,Reason Joined,Reinstate Date,Renew Method,ZIP,Mosaic Household,Mosaic Global Household,kcl_B_IND_MosaicsGrouping,New Mover Flag,Occupation Code,Occupation Group,Right_Dwelling Type,Move Distance,Occupant Type,Breakdown Map Location,Breakdown City,Breakdown State,Basic Cost,Calculated Tow Miles,Call Canceled,Call Killed,Call Status Recv Date,Cash Call,Clearing Code Last Description,Dispatch Code1 Description,Dispatch Code2Description,DTL Prob1 Code Description,Fleet Indicator,Is Duplicate,Is NSR,Member Match Flag,Member Number and Associate ID,Motorcycle Indicator,Plus Cost,Plus Indicator Description,Premier Cost,Prob1 Code Description,Prob2 Code Description,SC Call Club Code Description,SC Date,Rec ID,SC STS RSN Code Description,SC Vehicle Manufacturer Name,SC Vehicle Model Name,SVC Facility Name,SVC Facility Type,Total Cost,Tow Destination Latitude,Tow Destination Longitude,Tow Destination Name,Was Duplicated,Was Towed To AAR Referral
0,0,100000030,104625900,Y,NEW HAVEN,CT,65110,651113490,N,N,N,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,Small or large multi-family w/apt number,NaN,NaN,Male,NaT,NaT,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,522115500,45007910,Y,WEST WARWICK,RI,28930,289338500,N,Y,N,N,N,N,N,N,One Child,NaN,Caucasion / White - English,150.0,Yes,Home Owner,"30-39,999",1922-02-05,Yes,Some College,Small or large multi-family w/apt number,600-649,English,NaN,2020-09-01,2005-01-28 12:41:09,F,New Member,1922-02-05,RI - WARWICK BRANCH,NaT,NaN,KENT,0.0,0.0,NaN,0.0,0.0,20.0,0.0,0.0,650.0,NaN,522115500.0,1970-09-01,1970-09-01,15000140.0,"{41.696,-71.5308}",153000.0,VoIP,ACTIVE,490.0,Primary,1530.0,NaN,Opt-In,NaN,2019-08-23,AUTO RENEW,28930.0,Senior Discounts,Low Income Elders,Golden Year Guardians,N,NaN,NaN,NaN,NaN,NaN,"{41,-71}",West Warwick,RI,325.0,8.0,N,N,2017-04-13 10:05:17,Y,CASH CALL ONLY C,Engine Overheat,NaN,Engine Overheat,N,0.0,0.0,10.0,153000.0,N,0.0,Basic Membership,0.0,Tow,NaN,AAA Northeast,2017-04-13,97073200.0,CASH CALL ONLY C,TOYOTA,CAMRY,ASTRO WRECKER SERVICE,independent repair,325.0,410.0,-710.0,Aar / Johnson's Auto Service,0.0,10.0


* General Info

In [8]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21344 entries, 0 to 21343
Columns: 113 entries, Column1 to Was Towed To AAR Referral
dtypes: datetime64[ns](9), float64(31), int64(5), object(68)
memory usage: 18.4+ MB


In [9]:
df_original.dtypes;

# Data processing 

* Creating Dictionary to keep track of each operation and each filtering per operation

In [10]:
removed_columns = {}
removed_rows = {}

## 1 - Dropping Erroneous Column (excel did that on converting the csv original file)

In [11]:
df_original.drop(columns='Column1', inplace=True)

In [12]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21344 entries, 0 to 21343
Columns: 112 entries, Individual Key to Was Towed To AAR Referral
dtypes: datetime64[ns](9), float64(31), int64(4), object(68)
memory usage: 18.2+ MB


## 2 - Grouping By Househol Key (main objective) and separate by variable type

    2.1 - Defining Dataframe Types 

In [13]:
binary_columns = ['FSV CMSI Flag', 'FSV Credit Card Flag', 'FSV Deposit Program Flag', 'FSV Home Equity Flag', 'FSV ID Theft Flag', 'FSV Mortgage Flag', 'INS Client Flag', 'TRV Globalware Flag', 'Responded to Catalog', 'Mail Responder', 'Home Owner', 'Children', 'Gender', 'Bad Address Flag', 'Do Not Direct Mail Solicit', 'Email Available', 'Opt-Out - Publication', 'New Mover Flag', 'Occupant Type', 'Call Canceled', 'Call Killed', 'Cash Call', 'Fleet Indicator', 'Is Duplicate', 'Is NSR',	'Member Match Flag', 'Was Duplicated',	'Was Towed To AAR Referral'] # binary data that will be threat different from the rest of the numerical and categorical data

binary = pd.DataFrame() # individual binary dataframe

binary['Household Key'] = df_original['Household Key'] # concatenating household key column to perform groupby method

numericals = pd.DataFrame() # individual numerical dataframe

string = pd.DataFrame() # individual string dataframe

string['Household Key'] = df_original['Household Key'] # concatenating household key column to perform groupby method

dates = pd.DataFrame() # individual dates dataframe

dates['Household Key'] = df_original['Household Key'] # concatenating household key column to perform groupby method

    2.2 - Filling numerical, binary numerical, and dates dataframe

In [14]:
for c in tqdm_notebook(df_original.columns, desc='Process Progress'): # for all the columns

    if c in binary_columns: # if the columns is in the binary categorie

        binary[c] = df_original[c] # fill in the binary dataframe
    
    elif df_original.dtypes[c] in ['int64', 'float64']: # if the variables type of the given column are float or iteger

        numericals[c] = df_original[c] # fill in the numerical dataframe

    elif df_original.dtypes[c] in ['<M8[ns]']: # if the variables type of the given column are in the data format
        
        dates[c] = df_original[c] # fill in the date dataframe

    else:

        string[c] = df_original[c] # else, fill in the string dataframe
    

In [15]:
numericals = numericals.groupby('Household Key').mean() # grouping by the mean of numerical values per each Household Key

binary_numericals = binary.groupby('Household Key').mean() # grouping by the mean of binary values per each Household Key We still need to separate categorical and numerical binary variables to perform the groupby procedure in the categorical columns

In [16]:
for c in tqdm_notebook(binary.columns, desc='Process Progress'): # Separating Binary Types
    if (c != 'Household Key'): # we dont want to iterate over the Household Key unecessarily
        if (binary.dtypes[c] in ['int64', 'float64']): # if the variables type of the given column are float or iteger 

            binary.drop(columns=c, inplace=True) # dropped from the future categorical binary dataframe

In [17]:
for c in tqdm_notebook(dates.columns, desc='Process Progress'): # Converting Dates into Years
    if (c != 'Household Key'):
        dates[c] = dates[c].astype('datetime64[ns]')
        dates[c] = dates[c].dt.year
        #dates[c] = dates[c].astype('int64')

In [18]:
dates  = dates.groupby('Household Key').mean() # grouping by the mean of year values per each Household Key

    2.3 - Strings dataframe

In [19]:
string = string.groupby('Household Key').apply(lambda x: x.mode()) # grouping by the mode of each categorical per each Household Key

In [20]:
string.head(n=2) # by some reason, the groupby method by the mode of string columns returned a multi-index dataframe. Therefore, we need to remove the multi-index of this dataframe to move further our analysis

Household Key Member Flag      City State - Grouped  \
Household Key                                                          
8750          0         8750.0           Y  CRANSTON              RI   
              1            NaN         NaN       NaN             NaN   

                Number of Children Race Income Education Dwelling Type  \
Household Key                                                            
8750          0                NaN  NaN    NaN       NaN          SFDU   
              1                NaN  NaN    NaN       NaN           NaN   

                Credit Ranges Language Billing Code Description  \
Household Key                                                     
8750          0           NaN      NaN      Renewal Mbr Billing   
              1           NaN      NaN                      NaN   

                          Branch Name Cancel Reason      County  Email Status  \
Household Key                                                                   
8750          0  RI - CRANSTON BRANCH   Non-Payment  PROVIDENCE        Active   
              1                   NaN           NaN         NaN  Unsubscribed   

                Right_Gender Member Map Location Member Phone Type  \
Household Key                                                        
8750          0          NaN  {41.7658,-71.4978}          Wireless   
              1          NaN                 NaN               NaN   

                Member Status Member Type Reason Joined Renew Method  \
Household Key                                                          
8750          0        ACTIVE   Associate           NaN       NOTICE   
              1           NaN         NaN           NaN          NaN   

                   Mosaic Household Mosaic Global Household  \
Household Key                                                 
8750          0  Couples with Clout    Bourgeois Prosperity   
              1                 NaN                     NaN   

                kcl_B_IND_MosaicsGrouping Occupation Code Occupation Group  \
Household Key                                                                
8750          0               Power Elite             NaN              NaN   
              1                       NaN             NaN              NaN   

                Right_Dwelling Type Move Distance Breakdown Map Location  \
Household Key                                                              
8750          0   Multi Family With  1 to 5 Miles               {41,-71}   
              1                 NaN           NaN                    NaN   

                Breakdown City Breakdown State Call Status Recv Date  \
Household Key                                                          
8750          0       Cranston              RI   2014-09-27 17:54:38   
              1            NaN             NaN   2014-09-29 06:34:41   

                Clearing Code Last Description   Dispatch Code1 Description  \
Household Key                                                                 
8750          0                 Normal Service            Flat Tire w/spare   
              1                            NaN  Transmission/Clutch Failure   

                  DTL Prob1 Code Description Motorcycle Indicator  \
Household Key                                                       
8750          0            Flat Tire w/spare                    N   
              1  Transmission/Clutch Failure                  NaN   

                Plus Indicator Description Prob1 Code Description  \
Household Key                                                       
8750          0            Plus Membership                    Tow   
              1                        NaN                    NaN   

                Prob2 Code Description SC Call Club Code Description  \
Household Key                                                          
8750          0                    NaN                 AAA Northeast   
              1     

        2.3.1 - Post-Processed Strings Dataframe

In [21]:
post_processed_string = pd.DataFrame() # creating a new dataframe for the post processed string variable

In [22]:
for house_id in tqdm_notebook(df_original['Household Key'].unique().tolist(), desc='Process Progress'): # iterating over all unique Household Keys

    if len(string.loc[house_id].index.tolist()) > 1: # if, for a given Household Key, we have more then one multi-index value

        result = string.loc[house_id].iloc[0] + string.loc[house_id].iloc[1] # we sum the first two rows

        non_null_indexes = np.where(string.loc[house_id].iloc[1].isnull().tolist())[0] # we evaluate which of them were not null before the sum procedure

        for i in np.where(string.loc[house_id].iloc[1].isnull().tolist())[0]: # we iterate over the indexes that were not null before the sum procedure

            result[i] = string.loc[house_id].iloc[0][i] # we replace by the previous non null value

        post_processed_string = pd.concat([post_processed_string, result.to_frame().T]) # and we store on our new dataframe

    else: # if, for a given Household Key, we have only one multi-index value

        result = string.loc[house_id].iloc[0] # we store it because we can not change it so far

        post_processed_string = pd.concat([post_processed_string, result.to_frame().T]) # # and we store on our new dataframe

In [23]:
post_processed_string = post_processed_string.set_index('Household Key') # set index to be sorted as Household Key values

post_processed_string.index = post_processed_string.index.astype(int) # converting the index to iterger

post_processed_string.head(n=3)

,Member Flag,City,State - Grouped,Number of Children,Race,Income,Education,Dwelling Type,Credit Ranges,Language,Billing Code Description,Branch Name,Cancel Reason,County,Email Status,Right_Gender,Member Map Location,Member Phone Type,Member Status,Member Type,Reason Joined,Renew Method,Mosaic Household,Mosaic Global Household,kcl_B_IND_MosaicsGrouping,Occupation Code,Occupation Group,Right_Dwelling Type,Move Distance,Breakdown Map Location,Breakdown City,Breakdown State,Call Status Recv Date,Clearing Code Last Description,Dispatch Code1 Description,DTL Prob1 Code Description,Motorcycle Indicator,Plus Indicator Description,Prob1 Code Description,Prob2 Code Description,SC Call Club Code Description,SC STS RSN Code Description,SC Vehicle Manufacturer Name,SC Vehicle Model Name,SVC Facility Name,SVC Facility Type,Tow Destination Name
104625900,Y,NEW HAVEN,CT,NaN,NaN,NaN,NaN,Small or large multi-family w/apt number,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45007910,Y,WEST WARWICK,RI,One Child,Caucasion / White - English,"30-39,999",Some College,Small or large multi-family w/apt number,600-649,English,New Member,RI - WARWICK BRANCH,NaN,KENT,NaN,NaN,"{41.696,-71.5308}",VoIP,ACTIVE,Primary,NaN,AUTO RENEW,Senior Discounts,Low Income Elders,Golden Year Guardians,NaN,NaN,NaN,NaN,"{41,-71}",West Warwick,RI,2015-01-29 14:34:292015-09-25 16:26:14,Normal Service,Member Requests Battery Service,Member Requests Battery Service,N,Basic Membership,Battery Service,NaN,AAA Northeast,Normal Service,TOYOTA,CAMRY,ASTRO WRECKER SERVICEAstro Wrecker Service,independent repair,Aar / Johnson's Auto Service
43175160,Y,TIVERTON,RI,NaN,NaN,NaN,NaN,SFDU,NaN,NaN,Cancellation,RI - BARRINGTON BRANCH,Deceased,NEWPORT,NaN,NaN,"{41.673,-71.1886}",NaN,CANCELLED,Primary,NaN,NOTICE,Aging in Place,Comfortable Retirement,Autumn Years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


    2.4 - Binary Categorical Dataframe

In [24]:
binary_categoricals = binary.groupby('Household Key').apply(lambda x: x.mode()) # grouping by the mode of each categorical per each Household Key

In [25]:
binary_categoricals.head(n=5) # by some reason, the groupby method by the mode of binary_categoricals columns returned a multi-index dataframe. Therefore, we need to remove the multi-index of this dataframe to move further our analysis

Household Key FSV CMSI Flag FSV Credit Card Flag  \
Household Key                                                       
8750          0         8750.0             N                    N   
9690          0         9690.0             N                    N   
33380         0        33380.0             N                    N   
87180         0        87180.0             N                    N   
              1            NaN           NaN                  NaN   

                FSV Deposit Program Flag FSV Home Equity Flag  \
Household Key                                                   
8750          0                        N                    N   
9690          0                        N                    N   
33380         0                        N                    N   
87180         0                        N                    N   
              1                      NaN                  NaN   

                FSV ID Theft Flag FSV Mortgage Flag INS Client Flag  \
Household Key                                                         
8750          0                 N                 N               N   
9690          0                 N                 N               N   
33380         0                 N                 N               N   
87180         0                 N                 N               N   
              1               NaN               NaN             NaN   

                TRV Globalware Flag Responded to Catalog Mail Responder  \
Household Key                                                             
8750          0                   N                  NaN            NaN   
9690          0                   N                  NaN            Yes   
33380         0                   N                  NaN            Yes   
87180         0                   N                  NaN            Yes   
              1                   Y                  NaN            NaN   

                 Home Owner Children Gender Bad Address Flag  \
Household Key                                                  
8750          0         NaN      NaN    NaN                F   
9690          0  Home Owner       No   Male                F   
33380         0  Home Owner      Yes    NaN                F   
87180         0  Home Owner       No   Male                F   
              1         NaN      NaN    NaN              NaN   

                Opt-Out - Publication New Mover Flag Occupant Type  \
Household Key                                                        
8750          0                Opt-In              N        Renter   
9690          0                Opt-In              N           NaN   
33380         0                Opt-In              N           NaN   
87180         0                Opt-In              N           NaN   
              1                   NaN            NaN           NaN   

                Call Canceled Call Killed Cash Call Fleet Indicator  
Household Key                                                        
8750          0             N           N         N               Y  
9690          0             N           N         N               Y  
33380         0           NaN         NaN       NaN             NaN  
87180         0           NaN         NaN       NaN             NaN  
              1           NaN         NaN       NaN             NaN

        2.4.1 - Post-Processed Binary Categorical Dataframe

In [26]:
post_processed_binary_categoricals = pd.DataFrame() # creating a new dataframe for the post processed string variable

In [27]:
for house_id in tqdm_notebook(df_original['Household Key'].unique().tolist(), desc='Process Progress'): # iterating over all unique Household Keys

    if len(binary_categoricals.loc[house_id].index.tolist()) > 1: # if, for a given Household Key, we have more then one multi-index value

        result = binary_categoricals.loc[house_id].iloc[0] + binary_categoricals.loc[house_id].iloc[1] # we sum the first two rows

        non_null_indexes = np.where(binary_categoricals.loc[house_id].iloc[1].isnull().tolist())[0] # we evaluate which of them were not null before the sum procedure

        for i in np.where(binary_categoricals.loc[house_id].iloc[1].isnull().tolist())[0]: # we iterate over the indexes that were not null before the sum procedure

            result[i] = binary_categoricals.loc[house_id].iloc[0][i] # we replace by the previous non null value

        post_processed_binary_categoricals = pd.concat([post_processed_binary_categoricals, result.to_frame().T]) # and we store on our new dataframe

    else: # if, for a given Household Key, we have only one multi-index value

        result = binary_categoricals.loc[house_id].iloc[0] # we store it because we can not change it so far

        post_processed_binary_categoricals = pd.concat([post_processed_binary_categoricals, result.to_frame().T]) # # and we store on our new dataframe

In [28]:
post_processed_binary_categoricals = post_processed_binary_categoricals.set_index('Household Key') # set index to be sorted as Household Key values

post_processed_binary_categoricals.index = post_processed_binary_categoricals.index.astype(int) # converting the index to iterger

post_processed_binary_categoricals.head(n=2)

,FSV CMSI Flag,FSV Credit Card Flag,FSV Deposit Program Flag,FSV Home Equity Flag,FSV ID Theft Flag,FSV Mortgage Flag,INS Client Flag,TRV Globalware Flag,Responded to Catalog,Mail Responder,Home Owner,Children,Gender,Bad Address Flag,Opt-Out - Publication,New Mover Flag,Occupant Type,Call Canceled,Call Killed,Cash Call,Fleet Indicator
104625900,N,N,N,N,N,N,N,N,NaN,NaN,NaN,NaN,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45007910,N,Y,N,N,N,N,N,N,NaN,Yes,Home Owner,Yes,NaN,F,Opt-In,N,NaN,N,N,N,N


In [29]:
print(len(numericals.index), len(binary_numericals.index), len(post_processed_binary_categoricals.index), len(dates.index), len(post_processed_string.index)) # number of rows per type of variable

5241 5241 5241 5241 5241


In [30]:
print(len(numericals.columns), len(binary_numericals.columns), len(post_processed_binary_categoricals.columns), len(dates.columns), len(post_processed_string.columns)) # number of columns per type of variable

27 7 21 9 47


## 3 - Evaluating number of null values in Columns (Data Processing)

    3.1 - Calculating null values ratio

In [31]:
columns_nullval_ratio = {} # dictionary of null values ration for all columns

sequence = ['Numerical', 'Binary Numericals', 'Dates', 'Binary Categoricals', 'Strings'] # key sequence for dictionaries

data_type_collection = [numericals, binary_numericals, dates, post_processed_binary_categoricals, post_processed_string] # list of dataframes per each data type

for var_type, idx in zip(sequence, range(len(sequence))): 

    columns_nullval_ratio[var_type] = (data_type_collection[idx].isnull().sum() * 100 / len(data_type_collection[idx])).sort_values(ascending=False) # filling dataframe dictionary

plt.figure(figsize=(10,5))
sns.barplot(x=columns_nullval_ratio[sequence[0]].index, y=columns_nullval_ratio[sequence[0]])
plt.xticks(rotation='90')
plt.xlabel('Variables', fontsize=12, weight='bold')
plt.ylabel('Percentage of missing values', fontsize=12, weight='bold')
plt.title('Percentage of missing values for the' + ' ' + sequence[0] + ' ' + 'Dataframe', fontsize=15, weight='bold')
plt.savefig('percentage of missing values.png', dpi=200, bbox_inches='tight', pad_inches=0)
plt.show()

plt.figure(figsize=(10,5))

sns.barplot(x=columns_nullval_ratio[sequence[1]].index, y=columns_nullval_ratio[sequence[1]])
plt.xticks(rotation='90')
plt.xlabel('Variables', fontsize=10)
plt.ylabel('Percentage of missing values', fontsize=10)
plt.title('Percentage of missing values for the' + ' ' + sequence[1] + ' ' + 'Dataframe', fontsize=15)

plt.figure(figsize=(10,5))

sns.barplot(x=columns_nullval_ratio[sequence[2]].index, y=columns_nullval_ratio[sequence[2]])
plt.xticks(rotation='90')
plt.xlabel('Variables', fontsize=10)
plt.ylabel('Percentage of missing values', fontsize=10)
plt.title('Percentage of missing values for the' + ' ' + sequence[2] + ' ' + 'Dataframe', fontsize=15)

plt.figure(figsize=(10,5))

sns.barplot(x=columns_nullval_ratio[sequence[3]].index, y=columns_nullval_ratio[sequence[3]])
plt.xticks(rotation='90')
plt.xlabel('Variables', fontsize=10)
plt.ylabel('Percentage of missing values', fontsize=10)
plt.title('Percentage of missing values for the' + ' ' + sequence[3] + ' ' + 'Dataframe', fontsize=15)

plt.figure(figsize=(10,5))

sns.barplot(x=columns_nullval_ratio[sequence[4]].index, y=columns_nullval_ratio[sequence[4]])
plt.xticks(rotation='90')
plt.xlabel('Variables', fontsize=10)
plt.ylabel('Percentage of missing values', fontsize=10)
plt.title('Percentage of missing values for the' + ' ' + sequence[4] + ' ' + 'Dataframe', fontsize=15)

    3.2 - First checkpoint

In [32]:
checkpoint_1 = {} # creating dictionary for the first checkpoint

for var_type, idx in zip(sequence, range(len(sequence))): 
    checkpoint_1[var_type] = data_type_collection[idx].copy() # filling it with all types of dataframes

    3.3 - Threshold Difinition

In [33]:
column_threshold = 40.0 # threshold of 40 %

    3.4 - Creating removed columns dictionary key

In [34]:
key2 = str(column_threshold) + '% Column Threshold Removal' # key for our track removal dictionary

removed_columns[key2] = [] # adding to the new key to the dictionary

 3.5 - Performing Filtering

In [35]:
for c in tqdm_notebook(df_original.columns, desc='Process Progress'): # for all columns on the original dataframe
    for var_type in sequence: # for all type of separatable dataframe 

        if c in checkpoint_1[var_type].columns: # if the column is in the list of columns of the given type of dataframe

            if columns_nullval_ratio[var_type][c] >= column_threshold: # we check if the same does not satisfies the threshold limit

                checkpoint_1[var_type].drop(columns=c, inplace=True) # in this case we remove the given column from the given type of dataframe
                
                removed_columns[key2].append(c) # and we add its name to the list of removed columns

print('Removed Columns after filtering process: ', removed_columns[key2])


Removed Columns after filtering process:  ['Responded to Catalog', 'Gender', 'Address Change Date', 'Right_Gender', 'Reason Joined', 'Reinstate Date', 'Occupation Code', 'Occupation Group', 'Right_Dwelling Type', 'Move Distance', 'Occupant Type', 'Dispatch Code2Description', 'Prob2 Code Description', 'Tow Destination Name']


In [36]:
for var_type, idx in zip(sequence, range(len(sequence))): # lets check the result now

    columns_nullval_ratio[var_type] = (checkpoint_1[var_type].isnull().sum() * 100 / len(checkpoint_1[var_type])).sort_values(ascending=False) # filling dataframe dictionary

plt.figure(figsize=(10,5))

sns.barplot(x=columns_nullval_ratio[sequence[0]].index, y=columns_nullval_ratio[sequence[0]])
plt.xticks(rotation='90')
plt.xlabel('Variables', fontsize=10)
plt.ylabel('Percentage of missing values', fontsize=10)
plt.title('Percentage of missing values for the' + ' ' + sequence[0] + ' ' + 'Dataframe', fontsize=15)

plt.figure(figsize=(10,5))

sns.barplot(x=columns_nullval_ratio[sequence[1]].index, y=columns_nullval_ratio[sequence[1]])
plt.xticks(rotation='90')
plt.xlabel('Variables', fontsize=10)
plt.ylabel('Percentage of missing values', fontsize=10)
plt.title('Percentage of missing values for the' + ' ' + sequence[1] + ' ' + 'Dataframe', fontsize=15)

plt.figure(figsize=(10,5))

sns.barplot(x=columns_nullval_ratio[sequence[2]].index, y=columns_nullval_ratio[sequence[2]])
plt.xticks(rotation='90')
plt.xlabel('Variables', fontsize=10)
plt.ylabel('Percentage of missing values', fontsize=10)
plt.title('Percentage of missing values for the' + ' ' + sequence[2] + ' ' + 'Dataframe', fontsize=15)

plt.figure(figsize=(10,5))

sns.barplot(x=columns_nullval_ratio[sequence[3]].index, y=columns_nullval_ratio[sequence[3]])
plt.xticks(rotation='90')
plt.xlabel('Variables', fontsize=10)
plt.ylabel('Percentage of missing values', fontsize=10)
plt.title('Percentage of missing values for the' + ' ' + sequence[3] + ' ' + 'Dataframe', fontsize=15)

plt.figure(figsize=(10,5))

sns.barplot(x=columns_nullval_ratio[sequence[4]].index, y=columns_nullval_ratio[sequence[4]])
plt.xticks(rotation='90')
plt.xlabel('Variables', fontsize=10)
plt.ylabel('Percentage of missing values', fontsize=10)
plt.title('Percentage of missing values for the' + ' ' + sequence[4] + ' ' + 'Dataframe', fontsize=15)

## 4 - Subjective Evaluation (Data Processing)

    4.1 - Rename FSV, INS, and TRV columns that are related to products purchase flags

In [37]:
matching_FSV = [s for s in list(checkpoint_1['Binary Categoricals'].columns) if "FSV" in s] # creating a list of the columns that have at least the FSV mnemonic
matching_INS = [s for s in list(checkpoint_1['Binary Categoricals'].columns) if "INS" in s] # creating a list of the columns that have at least the INS mnemonic
matching_TRV = [s for s in list(checkpoint_1['Binary Categoricals'].columns) if "TRV" in s] # creating a list of the columns that have at least the TRV mnemonic
final_matching = matching_FSV + matching_INS + matching_TRV # concatenating them all
print(final_matching)

['FSV CMSI Flag', 'FSV Credit Card Flag', 'FSV Deposit Program Flag', 'FSV Home Equity Flag', 'FSV ID Theft Flag', 'FSV Mortgage Flag', 'INS Client Flag', 'TRV Globalware Flag']


In [38]:
counter = 1 # counter flag

for c in final_matching:

    checkpoint_1['Binary Categoricals'].rename(columns={c : 'Purchased Product' + ' ' + str(counter)}, inplace=True) # renaming the columns

    counter = counter + 1 # index of the new product column name

In [39]:
checkpoint_1['Binary Categoricals'].head(n=2)

,Purchased Product 1,Purchased Product 2,Purchased Product 3,Purchased Product 4,Purchased Product 5,Purchased Product 6,Purchased Product 7,Purchased Product 8,Mail Responder,Home Owner,Children,Bad Address Flag,Opt-Out - Publication,New Mover Flag,Call Canceled,Call Killed,Cash Call,Fleet Indicator
104625900,N,N,N,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45007910,N,Y,N,N,N,N,N,N,Yes,Home Owner,Yes,F,Opt-In,N,N,N,N,N


    4.2 -  Removing columns that are not allowed or not ethical to work


        4.2.1 - Creating Dictionary Key

In [40]:
key3 = 'Unethical Columns' # creating key to removed columns

removed_columns[key3] = ['Race', 'Language', 'Gender'] # since we are talking about Households, the same can not be differentiate by gender

        4.2.2 - Performing Filtering

In [41]:
for c in tqdm_notebook(removed_columns[key3], desc='Process Progress'): # iterating over the given columns
    for var_type in sequence: # for each dataframe type
        if c in checkpoint_1[var_type]: # if the columns is in the dataframe

            checkpoint_1[var_type].drop(columns=c, inplace=True) # we dropped

print('Removed Columns after filtering process: ', removed_columns[key3])


Removed Columns after filtering process:  ['Race', 'Language', 'Gender']


    4.3 - Removing Columns that represent the same information in a different way (information redundancy)

        4.3.1 - Creating Dictionary Key

In [42]:
key4 = 'Redundancy Information Columns' # generating key for this filtering procedure

removed_columns[key4] = ['Individual Key', 'State - Grouped', 'ZIP5', 'ZIP9', 'Children', 'Birth Date MMDDYYYY', 'Cancel Date', 'County', 'Do Not Direct Mail Solicit', 'Right_Individual Key', 'Member Key', 'Member Number Associate ID', 'Membership ID', 'Reinstate Date', 'ZIP', 'Mosaic Household', 'kcl_B_IND_MosaicsGrouping', 'Breakdown State', 'Call Killed', 'Clearing Code Last Description', 'Dispatch Code1 Description','DTL Prob1 Code Description', 'Is Duplicate', 'Member Match Flag', 'Member Number and Associate ID', 'SC Date', 'Rec ID', 'SC STS RSN Code Description', 'SC Vehicle Model Name', 'SVC Facility Name', 'SVC Facility Type', 'Tow Destination Latitude', 'Tow Destination Longitude', 'Member Map Location', 'Dwelling Type', 'Email Status', 'Cancel Reason', 'Breakdown City', 'Call Status Recv Date', 'Occupation Group'] #  Comlumns ids

        4.3.2 - Performing Filtering

In [43]:
for c in tqdm_notebook(removed_columns[key4], desc='Process Progress'): # for each column in the redundancy columns list
    for var_type in sequence: # for each dataframe type
        if c in checkpoint_1[var_type]: # if the columns is in the given dataframe
            checkpoint_1[var_type].drop(columns=c, inplace=True) # we dropped

print('Removed Columns after filtering process: ', removed_columns[key4])


Removed Columns after filtering process:  ['Individual Key', 'State - Grouped', 'ZIP5', 'ZIP9', 'Children', 'Birth Date MMDDYYYY', 'Cancel Date', 'County', 'Do Not Direct Mail Solicit', 'Right_Individual Key', 'Member Key', 'Member Number Associate ID', 'Membership ID', 'Reinstate Date', 'ZIP', 'Mosaic Household', 'kcl_B_IND_MosaicsGrouping', 'Breakdown State', 'Call Killed', 'Clearing Code Last Description', 'Dispatch Code1 Description', 'DTL Prob1 Code Description', 'Is Duplicate', 'Member Match Flag', 'Member Number and Associate ID', 'SC Date', 'Rec ID', 'SC STS RSN Code Description', 'SC Vehicle Model Name', 'SVC Facility Name', 'SVC Facility Type', 'Tow Destination Latitude', 'Tow Destination Longitude', 'Member Map Location', 'Dwelling Type', 'Email Status', 'Cancel Reason', 'Breakdown City', 'Call Status Recv Date', 'Occupation Group']


In [44]:
for var_type in sequence:

    print(var_type + ':', len(checkpoint_1[var_type].columns)) # number of remaining columns for each dataframe type after the filtering process 

Numerical: 14
Binary Numericals: 4
Dates: 4
Binary Categoricals: 16
Strings: 19


    4.4 - Checking Columns with low percentage of variantional information (counting null values)

        4.4.1 - Creating Dictionay Key 

In [45]:
key5 = 'Low Variational Information Columns' # generating key for this filtering procedure

removed_columns[key5] = [] # generating empty list

        4.4.2 - Performing Filtering

In [46]:
for var_type in tqdm_notebook(sequence, desc='Process Progress'): # for every dataframe type
    for c in checkpoint_1[var_type].columns: # for every column in the given dataframe

        if checkpoint_1[var_type].nunique()[c] == 1: # we check if the values on the given column are all the same

            checkpoint_1[var_type].drop(columns=c, inplace=True) # if it is so, we drop the column

            removed_columns[key5].append(c) # and we fill the removed columns list
        
print('Removed Columns after filtering process: ', removed_columns[key5])


Removed Columns after filtering process:  ['Member Flag']


In [47]:
for var_type in sequence:

    print(var_type + ':', len(checkpoint_1[var_type].columns)) # number of remaining columns for each dataframe type after the filtering process 

Numerical: 14
Binary Numericals: 4
Dates: 4
Binary Categoricals: 16
Strings: 18


    4.5 - Removing Columns with Unnecessary Information (extremely subjective)

        4.5.1 - Creating Dictionary Key

In [48]:
key6 = 'Unnecessary Columns' # generating key for this filtering procedure

removed_columns[key6] = ['Address Change Date', 'Bad Address Flag', 'Billing Code Description', 'Join Club Date', 'Member Phone Type', 'Mosaic Global Household', 'New Mover Flag', 'Call Canceled', 'Is NSR', 'Member Type', 'Was Duplicated', 'Was Towed To AAR Referral', 'Branch Name', 'Member Map Location', 'Breakdown Map Location'] #  Comlumns ids

        4.5.2 - Performing Filtering

In [49]:
for c in tqdm_notebook(removed_columns[key6], desc='Process Progress'): # for each column in the unncessary columns list
    for var_type in sequence: # for each dataframe type

        if c in checkpoint_1[var_type]: # if the column is in the given dataframe
            checkpoint_1[var_type].drop(columns=c, inplace=True) # we dropped

print('Removed Columns after filtering process: ', removed_columns[key6])


Removed Columns after filtering process:  ['Address Change Date', 'Bad Address Flag', 'Billing Code Description', 'Join Club Date', 'Member Phone Type', 'Mosaic Global Household', 'New Mover Flag', 'Call Canceled', 'Is NSR', 'Member Type', 'Was Duplicated', 'Was Towed To AAR Referral', 'Branch Name', 'Member Map Location', 'Breakdown Map Location']


In [50]:
for var_type in sequence:

    print(var_type + ':', len(checkpoint_1[var_type].columns)) # number of remaining columns for each dataframe type after the filtering process 

Numerical: 14
Binary Numericals: 1
Dates: 3
Binary Categoricals: 13
Strings: 12


## 5 - Null value evaluation for each Household Key 

    5.1 - Removing rows (Household Keys) beyond given threshold

        5.1.1 - Global Mean of Null Values for all Household Keys available

In [51]:
rows_nullval_ratio = {} # dictionary of null values sum for all columns

for var_type in sequence: 

    rows_nullval_ratio[var_type] = (checkpoint_1[var_type].isnull().sum(axis=1).sort_values(ascending=False)) # filling dataframe dictionary

In [52]:
sum_num_null_rows = [] # list of sum of all null values per row

for var_type in sequence:

    sum_num_null_rows.append(round(rows_nullval_ratio[var_type].sum())) # filling list

total_mean = round(sum(sum_num_null_rows)/len(rows_nullval_ratio['Numerical'].index.tolist())) # general mean of null values per row

print('\n','Mean Number of Null values on Rows:', total_mean) 


 Mean Number of Null values on Rows: 8.0


        5.1.2 - Creating Key and Dictionary for this operation

In [53]:
removed_rows = {}

row_key1 = str(total_mean) + ' Number of Null per Row Threshold Removal' # key for our track removal dictionary

removed_rows[row_key1] = [] # adding to the new key to the dictionary

        5.1.3 - Global Mean of each Household Key available

In [54]:
house_mean = {} # global mean number of null values for each Household Key

for house_id in tqdm_notebook(checkpoint_1[sequence[0]].index.tolist(), desc='Process Progress'): # for all unique Household Keys on the original dataframe
    tmp = 0 # temporary value for each global mean of the given Household Key
    for var_type in sequence: # for all type of separatable dataframe 

        tmp += rows_nullval_ratio[var_type][house_id]

    house_mean[house_id] =  tmp.mean()

        5.1.4 - Perform Filtering

In [55]:
for house_id in tqdm_notebook(checkpoint_1[sequence[0]].index.tolist(), desc='Process Progress'): # for all unique Household Keys on the original dataframe
    for var_type in sequence: # for all type of separatable dataframe        
        if house_mean[house_id] >= (total_mean * 2): # we check if the same does not satisfies the threshold limit

            for var_type in sequence: # for all type of separatable dataframe 

                if house_id in checkpoint_1[var_type].index.tolist(): # check if Household Key has not been already droped to prevent error raise

                    checkpoint_1[var_type].drop(house_id, inplace=True) # drop the row related to the given Household Key in all dataframes type

                    removed_rows[row_key1].append(house_id) # and we add its name to the list of removed columns

print('Removed Rows after filtering process: ', removed_rows[row_key1])


Removed Rows after filtering process:  [208500, 208500, 208500, 208500, 208500, 253650, 253650, 253650, 253650, 253650, 374680, 374680, 374680, 374680, 374680, 949270, 949270, 949270, 949270, 949270, 1152890, 1152890, 1152890, 1152890, 1152890, 1153460, 1153460, 1153460, 1153460, 1153460, 1401850, 1401850, 1401850, 1401850, 1401850, 1434580, 1434580, 1434580, 1434580, 1434580, 1797300, 1797300, 1797300, 1797300, 1797300, 1956800, 1956800, 1956800, 1956800, 1956800, 2686980, 2686980, 2686980, 2686980, 2686980, 2690970, 2690970, 2690970, 2690970, 2690970, 2954970, 2954970, 2954970, 2954970, 2954970, 2956820, 2956820, 2956820, 2956820, 2956820, 2968010, 2968010, 2968010, 2968010, 2968010, 2977460, 2977460, 2977460, 2977460, 2977460, 3253520, 3253520, 3253520, 3253520, 3253520, 3530390, 3530390, 3530390, 3530390, 3530390, 3828020, 3828020, 3828020, 3828020, 3828020, 3830820, 3830820, 3830820, 3830820, 3830820, 3831090, 3831090, 3831090, 3831090, 3831090, 3836860, 3836860, 3836860, 3836860

In [56]:
for var_type, idx in zip(sequence, range(len(sequence))): # lets check the result now

    rows_nullval_ratio[var_type] = (checkpoint_1[var_type].isnull().sum(axis=1).sort_values(ascending=False)) # filling dataframe dictionary

In [57]:
for var_type in sequence:

    print(var_type + ':', len(checkpoint_1[var_type].index)) # total number of rows in each type of dataframe

Numerical: 4095
Binary Numericals: 4095
Dates: 4095
Binary Categoricals: 4095
Strings: 4095


In [58]:
for var_type in sequence:

    print(var_type + ':', len(checkpoint_1[var_type].columns)) # number of remaining columns for each dataframe type after the filtering process 

Numerical: 14
Binary Numericals: 1
Dates: 3
Binary Categoricals: 13
Strings: 12


## 6 - Summarize Information and Fill Nulls in the given columns 

    6.1 - Saving checkpoint 

In [59]:
checkpoint_2 = {} # creating dictionary for the second checkpoint

for var_type, idx in zip(sequence, range(len(sequence))): 
    checkpoint_2[var_type] = checkpoint_1[var_type].copy() # filling in with all dataframe types

    6.2 - Dealing first with the Numericals Dataframe

In [60]:
checkpoint_2[sequence[0]].head(n=3) # lets remember what variables and how do they behave on this dataframe

,Length Of Residence,ERS ENT Count Year 1,ERS ENT Count Year 2,ERS ENT Count Year 3,ERS Member Cost Year 1,ERS Member Cost Year 2,ERS Member Cost Year 3,Member Tenure Years,Months from Join to Cancel,Basic Cost,Calculated Tow Miles,Plus Cost,Premier Cost,Total Cost
Household Key,,,,,,,,,,,,,,
8750,NaN,7.857143,4.285714,7.142857,4623.928571,2522.142857,5472.0,289.444444,1577.5,1250.357143,80.142857,265.714286,0.0,1516.071429
9690,140.0,0.000000,6.666667,15.000000,0.000000,1470.833333,15059.5,353.333333,NaN,3072.500000,22.500000,127.500000,0.0,4347.500000
33380,50.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,373.333333,3120.0,NaN,NaN,NaN,NaN,NaN


        6.2.1 - Lets take a look at the Length Of Residence variable

In [61]:
checkpoint_2[sequence[0]]['Length Of Residence'].unique() # total number of null values for the Length Of Residence variable

array([         nan, 140.        ,  50.        , 150.        ,
       121.42857143, 116.66666667,  21.25      , 103.33333333,
       130.        ,  90.        ,  96.        ,  75.        ,
       105.        ,  80.        ,  27.85714286, 133.33333333,
        92.5       , 110.        ,  70.        , 108.33333333,
       100.        ,   0.        ,  20.        , 112.5       ,
       136.66666667,  10.        ,  60.        , 120.        ,
       141.25      , 126.66666667,  26.66666667, 165.        ,
       122.77777778, 114.28571429,  98.        ,  86.        ,
       196.66666667,  45.        ,  91.42857143, 142.72727273,
       200.        , 128.        ,  40.        , 138.57142857,
       152.85714286,  42.        ,  94.11764706, 106.42857143,
       137.14285714,  15.        , 121.66666667,  72.        ,
        42.5       ,  30.        , 124.        , 117.5       ,
       142.85714286, 132.5       , 122.5       ,  94.61538462,
       263.33333333, 290.        , 131.25      , 106.  

            6.2.2.1 - Round the values and converting them to interger type to prevent any error

In [62]:
for idx in tqdm_notebook(checkpoint_2[sequence[0]]['Length Of Residence'].index, desc='Process Progress'): # for all indexes of the given dataframe

    checkpoint_2[sequence[0]].at[idx, 'Length Of Residence'] = round(checkpoint_2[sequence[0]]['Length Of Residence'][idx]) # replace de Null value by the Household Key mean

    if np.isnan(checkpoint_2[sequence[0]]['Length Of Residence'][idx]):

        checkpoint_2[sequence[0]].at[idx, 'Length Of Residence'] = 0

# checkpoint_2[sequence[0]]['Length Of Residence'] = checkpoint_2[sequence[0]]['Length Of Residence'].astype(int)

In [63]:
Counter(checkpoint_2[sequence[0]]['Length Of Residence'].tolist());

    As we can see, we still have a considerable amount of null values in the Length Of Residence variable. If we replaced these values by the mean of the variable we would be disregarding the geographical relation. Therefore, we will use the mean related to the City variable to predict the null values

In [64]:
checkpoint_2['Strings']['City'].isnull().sum() # number of Household Keys with null City values

0

In [65]:
checkpoint_2[sequence[0]]['City'] = checkpoint_2['Strings']['City'].values # temporarily concatenate the city column on the numerical column

            6.2.2.2 - Luckly, we can rely on the City variable, stored at the Strings dataframe. Thus, we can proceed with the Null values prediction

In [66]:
for idx in tqdm_notebook(checkpoint_2[sequence[0]].index, desc='Process Progress'): # For all Household Keys

    if checkpoint_2[sequence[0]]['Length Of Residence'][idx] == 0: # we check if the Length of Residence variable is equal zero (previouly changed from nan values)

        tmp_city = checkpoint_2[sequence[0]]['City'][idx] # we store a temporary value of the city for the given Household Key

        checkpoint_2[sequence[0]]['Length Of Residence'][idx] = checkpoint_2[sequence[0]]['Length Of Residence'].loc[checkpoint_2[sequence[0]]['City'] == tmp_city].values.mean() # replacing zeroed value by mean of the Length of Residence per grouped by City

In [67]:
checkpoint_2[sequence[0]].head(n=2)

,Length Of Residence,ERS ENT Count Year 1,ERS ENT Count Year 2,ERS ENT Count Year 3,ERS Member Cost Year 1,ERS Member Cost Year 2,ERS Member Cost Year 3,Member Tenure Years,Months from Join to Cancel,Basic Cost,Calculated Tow Miles,Plus Cost,Premier Cost,Total Cost,City
Household Key,,,,,,,,,,,,,,,
8750,90.558333,7.857143,4.285714,7.142857,4623.928571,2522.142857,5472.0,289.444444,1577.5,1250.357143,80.142857,265.714286,0.0,1516.071429,WEST WARWICK
9690,140.000000,0.000000,6.666667,15.000000,0.000000,1470.833333,15059.5,353.333333,NaN,3072.500000,22.500000,127.500000,0.0,4347.500000,WARWICK


            6.2.2.3 - Droping City variable from Numerical dataframe type

In [68]:
checkpoint_2[sequence[0]].drop(columns='City', inplace=True)

        6.2.3 - Rename ERS ENT columns that are related to number of roadside calls and creating a column that presents the sum of it

            6.2.3.1 Finding given columns

In [69]:
matching_ERS_ENT = [s for s in list(checkpoint_2[sequence[0]].columns) if "ERS ENT" in s] # creating a list of the columns that have at least the ERS ENT mnemonic

print(matching_ERS_ENT)

['ERS ENT Count Year 1', 'ERS ENT Count Year 2', 'ERS ENT Count Year 3']


            6.2.3.2 Renaming Columns

In [70]:
counter = 1 # counter flag

for c in matching_ERS_ENT:

    checkpoint_2[sequence[0]].rename(columns={c : 'Number of Roadside calss for Year' + ' ' + str(counter)}, inplace=True) # renaming the columns

    counter = counter + 1 # index of the new product column name

In [71]:
checkpoint_2[sequence[0]].head(n=2)

,Length Of Residence,Number of Roadside calss for Year 1,Number of Roadside calss for Year 2,Number of Roadside calss for Year 3,ERS Member Cost Year 1,ERS Member Cost Year 2,ERS Member Cost Year 3,Member Tenure Years,Months from Join to Cancel,Basic Cost,Calculated Tow Miles,Plus Cost,Premier Cost,Total Cost
Household Key,,,,,,,,,,,,,,
8750,90.558333,7.857143,4.285714,7.142857,4623.928571,2522.142857,5472.0,289.444444,1577.5,1250.357143,80.142857,265.714286,0.0,1516.071429
9690,140.000000,0.000000,6.666667,15.000000,0.000000,1470.833333,15059.5,353.333333,NaN,3072.500000,22.500000,127.500000,0.0,4347.500000


            6.2.3.4 - Calculating Total Amount of Calls

In [72]:
new_column = [] # creating list for the new column to be appended

for idx in tqdm_notebook(checkpoint_2[sequence[0]].index, desc='Process Progress'): # For each row
    
    total_amount = 0 # total amount of calls for each Household Key

    for c in ['Number of Roadside calss for Year 1', 'Number of Roadside calss for Year 2',	'Number of Roadside calss for Year 3']: # for each Number of Roadside calls for each Year

        if np.isnan(checkpoint_2[sequence[0]][c][idx]): # checking for nan values existance

            checkpoint_2[sequence[0]].at[idx, c] = 0 # replacing them by zero calls
                  
        checkpoint_2[sequence[0]].at[idx, c] = round(checkpoint_2[sequence[0]][c][idx]) # rounding the call values

        total_amount += checkpoint_2[sequence[0]][c][idx] # summing the total amount of call

    new_column.append(total_amount) # appending to list representing the new column to be appended

In [73]:
for c in ['Number of Roadside calss for Year 1', 'Number of Roadside calss for Year 2',	'Number of Roadside calss for Year 3']: # for each Number of Roadside calls for each Year
    checkpoint_2[sequence[0]][c] = checkpoint_2[sequence[0]][c].astype(int) # converting the final values to iterger type


In [74]:
column_name = 'Total Number of Roadside Calls' # naming the new column

checkpoint_2[sequence[0]][column_name] = new_column # appendding new column

checkpoint_2[sequence[0]][column_name] = checkpoint_2[sequence[0]][column_name].astype(int) # converting to iterger type

In [75]:
checkpoint_2[sequence[0]].head(n=2)

,Length Of Residence,Number of Roadside calss for Year 1,Number of Roadside calss for Year 2,Number of Roadside calss for Year 3,ERS Member Cost Year 1,ERS Member Cost Year 2,ERS Member Cost Year 3,Member Tenure Years,Months from Join to Cancel,Basic Cost,Calculated Tow Miles,Plus Cost,Premier Cost,Total Cost,Total Number of Roadside Calls
Household Key,,,,,,,,,,,,,,,
8750,90.558333,8,4,7,4623.928571,2522.142857,5472.0,289.444444,1577.5,1250.357143,80.142857,265.714286,0.0,1516.071429,19
9690,140.000000,0,7,15,0.000000,1470.833333,15059.5,353.333333,NaN,3072.500000,22.500000,127.500000,0.0,4347.500000,22


        6.2.4 - Rename ERS Member that are related to the cost of roadside services per year and creating a column that presents the sum of it

            6.2.4.1 - Finding given columns

In [76]:
matching_ERS_Cost = [s for s in list(checkpoint_2[sequence[0]].columns) if "ERS Member" in s] # creating a list of the columns that have at least the ERS Member mnemonic

print(matching_ERS_Cost)

['ERS Member Cost Year 1', 'ERS Member Cost Year 2', 'ERS Member Cost Year 3']


            6.2.4.2 - Renaming Columns

In [77]:
counter = 1 # counter flag

for c in matching_ERS_Cost:

    checkpoint_2[sequence[0]].rename(columns={c : 'Roadside Member Cost for Year' + ' ' + str(counter)}, inplace=True) # renaming the columns

    counter = counter + 1 # index of the new product column name

In [78]:
checkpoint_2[sequence[0]].head(n=2)

,Length Of Residence,Number of Roadside calss for Year 1,Number of Roadside calss for Year 2,Number of Roadside calss for Year 3,Roadside Member Cost for Year 1,Roadside Member Cost for Year 2,Roadside Member Cost for Year 3,Member Tenure Years,Months from Join to Cancel,Basic Cost,Calculated Tow Miles,Plus Cost,Premier Cost,Total Cost,Total Number of Roadside Calls
Household Key,,,,,,,,,,,,,,,
8750,90.558333,8,4,7,4623.928571,2522.142857,5472.0,289.444444,1577.5,1250.357143,80.142857,265.714286,0.0,1516.071429,19
9690,140.000000,0,7,15,0.000000,1470.833333,15059.5,353.333333,NaN,3072.500000,22.500000,127.500000,0.0,4347.500000,22


            6.2.4.3 - Calculating Total Amount of Calls

In [79]:
new_column = [] # creating list for the new column to be appended

for idx in tqdm_notebook(checkpoint_2[sequence[0]].index, desc='Process Progress'): # For each row
    
    total_amount = 0 # total amount of Roadside cost for each Household Key

    for c in ['Roadside Member Cost for Year 1', 'Roadside Member Cost for Year 2',	'Roadside Member Cost for Year 3']: # for each cost of Roadside for each Year

        if np.isnan(checkpoint_2[sequence[0]][c][idx]): # checking for nan values existance

            checkpoint_2[sequence[0]].at[idx, c] = 0 # replacing them by zero cost
                  
        total_amount += checkpoint_2[sequence[0]][c][idx] # summing the total amount of roadside cost

    new_column.append(total_amount) # appending to list representing the new column to be appended

In [80]:
column_name = 'Total Roadside Member Cost' # naming the new column

checkpoint_2[sequence[0]][column_name] = new_column # appendding new column

In [81]:
checkpoint_2[sequence[0]].head(n=2)

,Length Of Residence,Number of Roadside calss for Year 1,Number of Roadside calss for Year 2,Number of Roadside calss for Year 3,Roadside Member Cost for Year 1,Roadside Member Cost for Year 2,Roadside Member Cost for Year 3,Member Tenure Years,Months from Join to Cancel,Basic Cost,Calculated Tow Miles,Plus Cost,Premier Cost,Total Cost,Total Number of Roadside Calls,Total Roadside Member Cost
Household Key,,,,,,,,,,,,,,,,
8750,90.558333,8,4,7,4623.928571,2522.142857,5472.0,289.444444,1577.5,1250.357143,80.142857,265.714286,0.0,1516.071429,19,12618.071429
9690,140.000000,0,7,15,0.000000,1470.833333,15059.5,353.333333,NaN,3072.500000,22.500000,127.500000,0.0,4347.500000,22,16530.333333


        6.2.5 - Converting Months to Join to Cancel to Years left for subscription to end

            6.2.5.1 - Renaming Column

In [82]:
checkpoint_2[sequence[0]].rename(columns={'Months from Join to Cancel' : 'Years to Cancelled Status'}, inplace=True) # renaming the column

            6.2.5.2 - Converting Months to Years

In [83]:
for idx in tqdm_notebook(checkpoint_2[sequence[0]].index, desc='Process Progress'): # For each row
    
    if ~np.isnan(checkpoint_2[sequence[0]]['Years to Cancelled Status'][idx]): # if the given Household Key does not have a null value for the given column
    
        checkpoint_2[sequence[0]].at[idx, 'Years to Cancelled Status'] = round(checkpoint_2[sequence[0]]['Years to Cancelled Status'][idx]/12) # converting months to years

            6.2.5.3 - Fill Null Values

In [84]:
mean_year = round(checkpoint_2[sequence[0]]['Years to Cancelled Status'].values[~np.isnan(checkpoint_2[sequence[0]]['Years to Cancelled Status'].values)].mean()) # mean year to cancelled status for all Household Keys
for idx in tqdm_notebook(checkpoint_2[sequence[0]].index, desc='Process Progress'): # For each row
    
        if np.isnan(checkpoint_2[sequence[0]]['Years to Cancelled Status'][idx]): # if the given Household Key does not have a null value for the given column

            checkpoint_2[sequence[0]].at[idx, 'Years to Cancelled Status'] = mean_year # replacing null values 
            
checkpoint_2[sequence[0]]['Years to Cancelled Status'] = checkpoint_2[sequence[0]]['Years to Cancelled Status'].astype(int) # converting to iterger type

In [85]:
checkpoint_2[sequence[0]].head(n=2)

,Length Of Residence,Number of Roadside calss for Year 1,Number of Roadside calss for Year 2,Number of Roadside calss for Year 3,Roadside Member Cost for Year 1,Roadside Member Cost for Year 2,Roadside Member Cost for Year 3,Member Tenure Years,Years to Cancelled Status,Basic Cost,Calculated Tow Miles,Plus Cost,Premier Cost,Total Cost,Total Number of Roadside Calls,Total Roadside Member Cost
Household Key,,,,,,,,,,,,,,,,
8750,90.558333,8,4,7,4623.928571,2522.142857,5472.0,289.444444,131,1250.357143,80.142857,265.714286,0.0,1516.071429,19,12618.071429
9690,140.000000,0,7,15,0.000000,1470.833333,15059.5,353.333333,242,3072.500000,22.500000,127.500000,0.0,4347.500000,22,16530.333333


        6.2.6 - Converting Member Tenure Years to integer type

In [86]:
for idx in tqdm_notebook(checkpoint_2[sequence[0]].index, desc='Process Progress'): # For each row
    
    checkpoint_2[sequence[0]].at[idx, 'Member Tenure Years'] = round(checkpoint_2[sequence[0]]['Member Tenure Years'][idx]) # rounding member tenure year

checkpoint_2[sequence[0]]['Member Tenure Years'] = checkpoint_2[sequence[0]]['Member Tenure Years'].astype(int) # converting to iterger type

In [87]:
checkpoint_2[sequence[0]].head(n=2)

,Length Of Residence,Number of Roadside calss for Year 1,Number of Roadside calss for Year 2,Number of Roadside calss for Year 3,Roadside Member Cost for Year 1,Roadside Member Cost for Year 2,Roadside Member Cost for Year 3,Member Tenure Years,Years to Cancelled Status,Basic Cost,Calculated Tow Miles,Plus Cost,Premier Cost,Total Cost,Total Number of Roadside Calls,Total Roadside Member Cost
Household Key,,,,,,,,,,,,,,,,
8750,90.558333,8,4,7,4623.928571,2522.142857,5472.0,289,131,1250.357143,80.142857,265.714286,0.0,1516.071429,19,12618.071429
9690,140.000000,0,7,15,0.000000,1470.833333,15059.5,353,242,3072.500000,22.500000,127.500000,0.0,4347.500000,22,16530.333333


        6.2.7 - Checking rest of the columns

        6.2.7.1 - Dealing with the rest of the Null values

In [88]:
for c in tqdm_notebook(checkpoint_2[sequence[0]].isnull().sum().index[checkpoint_2[sequence[0]].isnull().sum() != 0].tolist(), desc='Process Progress'): # for each column where there are still null values to treat
    for idx in checkpoint_2[sequence[0]].index: # For each Household ID 

        if np.isnan(checkpoint_1[sequence[0]][c][idx]): # we check if the previous checkpoint had this null value previously

            checkpoint_2[sequence[0]].at[idx, c] = 0 # we replace it for zero because we need to assume that 

        else:

            checkpoint_2[sequence[0]].at[idx, c] = checkpoint_1[sequence[0]][c][idx] # if it's not we replace with the previous value


plt.figure(figsize=(10,5))

sns.barplot(x=checkpoint_2[sequence[0]].isnull().sum().index, y=checkpoint_2[sequence[0]].isnull().sum())
plt.xticks(rotation='90')
plt.xlabel('Variables', fontsize=10)
plt.ylabel('Percentage of missing values', fontsize=10)
plt.title('Percentage of missing values for the' + ' ' + sequence[0] + ' ' + 'Dataframe', fontsize=15)

    6.3 - Moving to Binary Numericals

In [89]:
checkpoint_2[sequence[1]].head(n=2)

,Email Available
Household Key,
8750,3.333333
9690,8.333333


        6.3.1 - Converting Email Available to Binary (1-0) numerical 

In [90]:
for idx in tqdm_notebook(checkpoint_2[sequence[1]].index, desc='Process Progress'): # For each row
    for c in checkpoint_2[sequence[1]].columns:
        if checkpoint_2[sequence[1]][c][idx] != 0:
            checkpoint_2[sequence[1]].at[idx, c] = 1.0
            
        else:
            checkpoint_2[sequence[1]].at[idx, c] = 0.0
            
checkpoint_2[sequence[1]]['Email Available'] = checkpoint_2[sequence[1]]['Email Available'].astype(int) # converting to iterger type

In [91]:
checkpoint_2[sequence[1]].head(n=2)

,Email Available
Household Key,
8750,1
9690,1


        6.3.2 - Concatenating to Numericals 

In [92]:
checkpoint_2[sequence[0]]['Email Available'] = checkpoint_2[sequence[1]]['Email Available'].values # concatenate the Email Available column on the numerical column

    6.4 - Moving to Dates

In [93]:
checkpoint_2[sequence[2]].head(n=2)

,Date Of Birth,Active Expiration Date,Join AAA Date
Household Key,,,
8750,NaN,2018.333333,1990.055556
9690,1943.5,2020.000000,1983.666667


        6.4.1 - Checking Null values

plt.figure(figsize=(10,5))

sns.barplot(x=checkpoint_2[sequence[2]].isnull().sum().index, y=checkpoint_2[sequence[2]].isnull().sum())
plt.xticks(rotation='90')
plt.xlabel('Variables', fontsize=10)
plt.ylabel('Percentage of missing values', fontsize=10)
plt.title('Percentage of missing values for the' + ' ' + sequence[0] + ' ' + 'Dataframe', fontsize=15)

        6.4.2 - Replacing Null Date of Births to mean of the dataframe 

In [94]:
mean_date_birth = round(checkpoint_2[sequence[2]]['Date Of Birth'].values[~np.isnan(checkpoint_2[sequence[2]]['Date Of Birth'].values)].mean()) # mean date of birth for all Household Keys

for idx in tqdm_notebook(checkpoint_2[sequence[2]].index, desc='Process Progress'): # For each row
    for c in checkpoint_2[sequence[2]].columns: # For all columns on the given dataframe
    
        if np.isnan(checkpoint_2[sequence[2]][c][idx]): # if the given Household Key has a null value for the given column

            checkpoint_2[sequence[2]].at[idx, c] = mean_date_birth # replacing null values

        else:

            checkpoint_2[sequence[2]].at[idx, c] = round(checkpoint_2[sequence[2]][c][idx]) # rounding the year values

            6.4.3 - Converting all columns to iterger and Years number

In [95]:
for idx in tqdm_notebook(checkpoint_2[sequence[2]].index, desc='Process Progress'): # For each row
    for c in checkpoint_2[sequence[2]].columns: # For all columns on the given dataframe

        checkpoint_2[sequence[2]].at[idx, c] = abs(2020 - checkpoint_2[sequence[2]][c][idx]) # replacing by the number of the year

        checkpoint_2[sequence[2]][c] = checkpoint_2[sequence[2]][c].astype(int) # converting to iterger type

            6.4.4 - Renaming columns for better identification

In [96]:
checkpoint_2[sequence[2]].rename(columns={'Date Of Birth' : 'Age', 'Active Expiration Date': 'Years from most recent Membership Expiration', 'Join AAA Date': 'Years from Join Date'}, inplace=True) # renaming the column

In [97]:
checkpoint_2[sequence[2]].head()

,Age,Years from most recent Membership Expiration,Years from Join Date
Household Key,,,
8750,74,2,30
9690,76,0,36
33380,90,11,38
87180,68,3,11
115240,50,3,37


            6.4.5 - Appending new dataframe to numerical one

In [98]:
for c in checkpoint_2[sequence[2]].columns: # For all columns on the given dataframe

    checkpoint_2[sequence[0]][c] = checkpoint_2[sequence[2]][c].values # concatenate the columns on the numerical dataframe

In [99]:
checkpoint_2[sequence[0]].head()

,Length Of Residence,Number of Roadside calss for Year 1,Number of Roadside calss for Year 2,Number of Roadside calss for Year 3,Roadside Member Cost for Year 1,Roadside Member Cost for Year 2,Roadside Member Cost for Year 3,Member Tenure Years,Years to Cancelled Status,Basic Cost,Calculated Tow Miles,Plus Cost,Premier Cost,Total Cost,Total Number of Roadside Calls,Total Roadside Member Cost,Email Available,Age,Years from most recent Membership Expiration,Years from Join Date
Household Key,,,,,,,,,,,,,,,,,,,,
8750,90.558333,8,4,7,4623.928571,2522.142857,5472.0,289,131,1250.357143,80.142857,265.714286,0.0,1516.071429,19,12618.071429,1,74,2,30
9690,140.000000,0,7,15,0.000000,1470.833333,15059.5,353,242,3072.500000,22.500000,127.500000,0.0,4347.500000,22,16530.333333,1,76,0,36
33380,50.000000,0,0,0,0.000000,0.000000,0.0,373,260,0.000000,0.000000,0.000000,0.0,0.000000,0,0.000000,0,90,11,38
87180,150.000000,0,0,0,0.000000,0.000000,0.0,100,2,0.000000,0.000000,0.000000,0.0,0.000000,0,0.000000,1,68,3,11
115240,121.000000,3,22,10,1961.666667,12205.833333,530.0,363,93,5885.000000,12.000000,0.000000,0.0,5885.000000,35,14697.500000,1,50,3,37


    6.5 - Moving to Binary Categoricals

In [100]:
checkpoint_2[sequence[3]].head()

,Purchased Product 1,Purchased Product 2,Purchased Product 3,Purchased Product 4,Purchased Product 5,Purchased Product 6,Purchased Product 7,Purchased Product 8,Mail Responder,Home Owner,Opt-Out - Publication,Cash Call,Fleet Indicator
45007910,N,Y,N,N,N,N,N,N,Yes,Home Owner,Opt-In,N,N
116229910,N,N,N,N,N,N,N,N,Yes,Home Owner,Opt-In,N,Y
5798100,N,N,N,N,N,N,Y,N,Yes,Home Owner,Opt-In,N,N
71870170,N,N,N,N,N,N,N,N,Yes,Home Owner,Opt-In,N,Y
77280880,N,N,N,N,N,N,N,N,Yes,Home Owner,Opt-In,N,Y


        6.5.1 - Checking Null values

plt.figure(figsize=(10,5))

sns.barplot(x=checkpoint_2[sequence[3]].isnull().sum().index, y=checkpoint_2[sequence[3]].isnull().sum())
plt.xticks(rotation='90')
plt.xlabel('Variables', fontsize=10)
plt.ylabel('Percentage of missing values', fontsize=10)
plt.title('Percentage of missing values for the' + ' ' + sequence[3] + ' ' + 'Dataframe', fontsize=15)

        6.5.2 - Product Columns and creating column of total purchased itens

In [101]:
prod_cols = ['Purchased Product 1',	'Purchased Product 2',	'Purchased Product 3',	'Purchased Product 4',	'Purchased Product 5',	'Purchased Product 6',	'Purchased Product 7',	'Purchased Product 8'] # column IDs

checkpoint_2[sequence[3]]['Total Number of Purchased Products'] = 0 # total number of purchased products columns

checkpoint_2[sequence[3]]['Household Purchased Product'] = 0 # flag whether the given Household Key has bought a product on the last 3 years

        6.5.3 - Mail Option Information

In [102]:
mail_var = ['Mail Responder', 'Opt-Out - Publication'] # mail columns

checkpoint_2[sequence[3]]['Email Correspondent'] = 0 # new mail information column

        6.5.4 - Creating Columns for Home information

In [103]:
checkpoint_2[sequence[3]]['Home Renter'] = 0 # Household rents a home

checkpoint_2[sequence[3]]['No Home Ownership Info'] = 0 # no info about it

        6.5.5 - Converting categoricals to numericals

In [104]:
for idx in tqdm_notebook(checkpoint_2[sequence[3]].index, desc='Process Progress'): # For each row
    for c in checkpoint_2[sequence[3]].columns: # For all columns on the given dataframe

        possible_values = list(Counter(checkpoint_2[sequence[3]][c].values).keys()) # list of possible values for the given column

        if ('N' in possible_values) or ('Y' in possible_values) or ('NY' in possible_values): # if any possible 'Y', 'N', or 'NY' (sum of both possible mode answers) is on the given column

            if (checkpoint_2[sequence[3]][c][idx] == 'N'): # if the given Household Key has a null value or 'N' for the given column

                checkpoint_2[sequence[3]].at[idx, c] = 0 # we replace it by zero

            else:

                checkpoint_2[sequence[3]].at[idx, c] = 1 # we replace it by zero

        elif ('Yes' in possible_values) or ('No' in possible_values) or ('NoYes' in possible_values) or ('Opt-In' in possible_values) or ('Opt-Out' in possible_values): # if any possible 'Yes', 'No', 'NoYes', 'Opt-In', or 'Opt-Out' is on the given column --> in this case we have entered in the Mail Responder column or Opt-Out - Publication

            if (checkpoint_2[sequence[3]][c][idx] == 'Yes') or (checkpoint_2[sequence[3]][c][idx] == 'NoYes') or (checkpoint_2[sequence[3]][c][idx] == 'Opt-In'):

                checkpoint_2[sequence[3]].at[idx, c] = 1

                checkpoint_2[sequence[3]].at[idx, 'Email Correspondent'] = 1

            else:

                if pd.isnull(checkpoint_2[sequence[3]][c][idx]): # if the given Household Key has a null value for the Mail Responder column

                    checkpoint_2[sequence[3]].at[idx, c] = checkpoint_2[sequence[0]]['Email Available'][idx] # we resort to the Email Available column to predict the null value for the given Household Key

                else:
                    
                    checkpoint_2[sequence[3]].at[idx, c] = 0

        elif ('Home Owner' in possible_values) or ('Probable Home Owner' in possible_values) or ('Probable Renter' in possible_values) or ('Renter' in possible_values): # if any possible 'Home Owner', 'Probable Home Owner', 'Probable Renter', or 'Renter' (sum of both possible mode answers) is on the given column --> in this case we have entered in the Mail Responder column

            if (checkpoint_2[sequence[3]][c][idx] == 'Home Owner') or (checkpoint_2[sequence[3]][c][idx] == 'Probable Home Owner'):

                checkpoint_2[sequence[3]].at[idx, c] = 1

            else:

                if pd.isnull(checkpoint_2[sequence[3]][c][idx]): # if the given Household Key has a null value for the Mail Responder column

                    checkpoint_2[sequence[3]].at[idx, 'No Home Ownership Info'] = 1

                    checkpoint_2[sequence[3]].at[idx, c] = 0

                else:
                    
                    checkpoint_2[sequence[3]].at[idx, c] = 0

                    checkpoint_2[sequence[3]].at[idx, 'Home Renter'] = 1

In [105]:
for c in checkpoint_2[sequence[3]].columns: # For all columns on the given dataframe

    checkpoint_2[sequence[3]][c] = checkpoint_2[sequence[3]][c].astype(int) # converting to iterger type

        6.5.6 - Filling Total Number of Purchased Products

In [106]:
new_values = []

for idx in tqdm_notebook(checkpoint_2[sequence[3]].index, desc='Process Progress'): # For each row
    
    total_amount = 0 # total amount of products bought in each row

    for c in prod_cols: # In each Purchased Product Column

        if checkpoint_2[sequence[3]][c][idx] == 1:

            checkpoint_2[sequence[3]].at[idx, 'Household Purchased Product'] = 1

            total_amount = total_amount + 1

    new_values.append(total_amount)

checkpoint_2[sequence[3]]['Total Number of Purchased Products'] = new_values

        6.5.7 - Appending columns to numerical one

In [107]:
for c in checkpoint_2[sequence[3]].columns: # For all columns on the given dataframe

    checkpoint_2[sequence[0]][c] = checkpoint_2[sequence[3]][c].values # concatenate the columns on the numerical dataframe

In [108]:
checkpoint_2[sequence[0]].head()

,Length Of Residence,Number of Roadside calss for Year 1,Number of Roadside calss for Year 2,Number of Roadside calss for Year 3,Roadside Member Cost for Year 1,Roadside Member Cost for Year 2,Roadside Member Cost for Year 3,Member Tenure Years,Years to Cancelled Status,Basic Cost,Calculated Tow Miles,Plus Cost,Premier Cost,Total Cost,Total Number of Roadside Calls,Total Roadside Member Cost,Email Available,Age,Years from most recent Membership Expiration,Years from Join Date,Purchased Product 1,Purchased Product 2,Purchased Product 3,Purchased Product 4,Purchased Product 5,Purchased Product 6,Purchased Product 7,Purchased Product 8,Mail Responder,Home Owner,Opt-Out - Publication,Cash Call,Fleet Indicator,Total Number of Purchased Products,Household Purchased Product,Email Correspondent,Home Renter,No Home Ownership Info
Household Key,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8750,90.558333,8,4,7,4623.928571,2522.142857,5472.0,289,131,1250.357143,80.142857,265.714286,0.0,1516.071429,19,12618.071429,1,74,2,30,0,1,0,0,0,0,0,0,1,1,1,0,0,1,1,1,0,0
9690,140.000000,0,7,15,0.000000,1470.833333,15059.5,353,242,3072.500000,22.500000,127.500000,0.0,4347.500000,22,16530.333333,1,76,0,36,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,1,0,0
33380,50.000000,0,0,0,0.000000,0.000000,0.0,373,260,0.000000,0.000000,0.000000,0.0,0.000000,0,0.000000,0,90,11,38,0,0,0,0,0,0,1,0,1,1,1,0,0,1,1,1,0,0
87180,150.000000,0,0,0,0.000000,0.000000,0.0,100,2,0.000000,0.000000,0.000000,0.0,0.000000,0,0.000000,1,68,3,11,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,1,0,0
115240,121.000000,3,22,10,1961.666667,12205.833333,530.0,363,93,5885.000000,12.000000,0.000000,0.0,5885.000000,35,14697.500000,1,50,3,37,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,1,0,0


    6.6 - Finally, moving to String Data

In [109]:
checkpoint_2[sequence[4]].head()

,City,Number of Children,Income,Education,Credit Ranges,Member Status,Renew Method,Motorcycle Indicator,Plus Indicator Description,Prob1 Code Description,SC Call Club Code Description,SC Vehicle Manufacturer Name
45007910,WEST WARWICK,One Child,"30-39,999",Some College,600-649,ACTIVE,AUTO RENEW,N,Basic Membership,Battery Service,AAA Northeast,TOYOTA
116229910,WARWICK,No children,"50-59,999",Unknown,750-799,ACTIVE,AUTO RENEW,N,Basic Membership,Flat Tire,AAA Northeast,SUBARU
5798100,CENTRAL FALLS,No children,"50-59,999",Some College,650-699,ACTIVE,NOTICE,N,Basic Membership,Lockout,AAA Northeast,INFINITI
71870170,WARWICK,One Child,"100-149,999",Unknown,650-699,PENDING,NOTICE,N,Basic Membership,Jump Start,AAA Northeast,VOLKSWAGEN
77280880,BARRINGTON,Four ChildrenThree Children,"100-149,999",Unknown,750-799800+,ACTIVE,NOTICE,N,Basic Membership,ExtricationFlat Tire,AAA Northeast,LEXUS


        6.6.1 - Checking Null values

plt.figure(figsize=(10,5))

sns.barplot(x=checkpoint_2[sequence[4]].isnull().sum().index, y=checkpoint_2[sequence[4]].isnull().sum())
plt.xticks(rotation='90')
plt.xlabel('Variables', fontsize=10)
plt.ylabel('Percentage of missing values', fontsize=10)
plt.title('Percentage of missing values for the' + ' ' + sequence[4] + ' ' + 'Dataframe', fontsize=15)

        6.6.2 - Performing Null filling and creating new identities

            6.6.2.1 - Number of Children

In [110]:
for idx in tqdm_notebook(checkpoint_2[sequence[4]].index, desc='Process Progress'): # For each row
    for c in checkpoint_2[sequence[4]].columns: # For all columns on the given dataframe

        if c == 'Number of Children': # if we the given column is the Number of Children

            if pd.isnull(checkpoint_2[sequence[4]][c][idx]): # if the given Household Key has a null value for the Number of Children column
            
                checkpoint_2[sequence[4]].at[idx, c] = 0 # we replace it by zero

            else:
                
                entry =  checkpoint_2[sequence[4]][c][idx].split()[0] # we check the first part of the string

                if (entry == 'No'):

                    checkpoint_2[sequence[4]].at[idx, c] = 0 # we replace it by zero, because there is no children for the given Household

                elif (entry == 'One'):

                    checkpoint_2[sequence[4]].at[idx, c] = 1 # we replace it by one, because there is one children for the given Household

                elif (entry == 'Two'):

                    checkpoint_2[sequence[4]].at[idx, c] = 2 # we replace it by two, because there are two children for the given Household

                elif (entry == 'Three'):

                    checkpoint_2[sequence[4]].at[idx, c] = 3 # we replace it by three, because there are three children for the given Household

                elif (entry == 'Four'):

                    checkpoint_2[sequence[4]].at[idx, c] = 4 # we replace it by four, because there are four children for the given Household

                elif (entry == 'Five'):

                    checkpoint_2[sequence[4]].at[idx, c] = 5 # we replace it by five, because there are five children for the given Household

                elif (entry == 'Six'):

                    checkpoint_2[sequence[4]].at[idx, c] = 6 # we replace it by six, because there are six children for the given Household

checkpoint_2[sequence[4]]['Number of Children'] = checkpoint_2[sequence[4]]['Number of Children'].astype(int) # converting to iterger type

In [111]:
for idx in tqdm_notebook(checkpoint_2[sequence[4]].index, desc='Process Progress'): # For each row
    for c in checkpoint_2[sequence[4]].columns: # For all columns on the given dataframe

        if (c == 'Credit Ranges'): # if we the given column is the Credit Ranges 

            if pd.isnull(checkpoint_2[sequence[4]][c][idx]): # if the given Household Key has a null value for Credit Ranges column
            
                checkpoint_2[sequence[4]].at[idx, c] = 499999 # we replace it by the lowest value of Credit Ranges possible

            else:

                item = checkpoint_2[sequence[4]][c][idx] # we save our string list

                if ("+" in item): # that means that we have the highest possible value of Credit Ranges on the given Household Key

                    checkpoint_2[sequence[4]].at[idx, c] = 800000 # so we set as the only the higher value

                elif ("&" in item) or ("Unk" in item): # that means that we have the lowest possible value of Credit Ranges on the given Household Key

                    if len(item.split('-')) == 1: # if we only have this value

                        checkpoint_2[sequence[4]].at[idx, c] = 499999 # we replace it by the lowest value of Credit Ranges possible

                    else: # if we have other values besides the lower Credit Ranges possible

                        lower_limit = 499999 # we set the lower limit by the lowest value of Credit Ranges possible

                        tmp = [] # we create an empty list

                        for s in item.split('-'): # we iterate over all values for this credit range
                            
                            tmp.append(s.split('Unk')[0]) # we append all numerical strings to it

                        tmp.sort() # we sort the list of numerical strings

                        upper_limit = float(tmp[-1]) * 1e3 # we choose the biggest value 

                        checkpoint_2[sequence[4]].at[idx, c] = (upper_limit + lower_limit)/2 # we replace the total value for the given Houlsehold Key by the mean

                else:

                        if len(item.split('-')) == 2: # if we only have two values for this range

                            lower_limit =  float(item.split('-')[0]) * 1e3 # lower Income limit

                            upper_limit = float(item.split('-')[1]) * 1e3 # Upper Income limit

                            checkpoint_2[sequence[4]].at[idx, c] = (upper_limit + lower_limit)/2 # we replace the total value for the given Household Key by the mean

                        else: # we have more than two values available

                            tmp = [] # we create an empty list

                            for s in item.split('-'): # we iterate over all values for this credit range
                            
                                tmp.append(s.split('Unk')[0].split(',')[0]) # we append all numerical strings to it

                            tmp.sort() # we sort the list of numerical strings

                            lower_limit = float(tmp[0]) * 1e3 # we choose the smaller value 
                                
                            upper_limit = float(tmp[-1]) * 1e3 # we choose the biggest value

                            checkpoint_2[sequence[4]].at[idx, c] = (upper_limit + lower_limit)/2 # we replace the total value for the given Houlsehold Key by the mean

checkpoint_2[sequence[4]]['Credit Ranges'] = checkpoint_2[sequence[4]]['Credit Ranges'].astype(float) # converting to float type

checkpoint_2[sequence[4]].rename(columns={'Credit Ranges' : 'Annual Credit Possible'}, inplace=True) # renaming the colum

    We will replace each limit by the mean of the given limit. The null values will be predicted as the lower values possible (For the Income is 9999.0 and for the Credit Range is 499999.0)

            6.6.2.2 - Income and Credit Ranges

In [112]:
for idx in tqdm_notebook(checkpoint_2[sequence[4]].index, desc='Process Progress'): # For each row
    for c in checkpoint_2[sequence[4]].columns: # For all columns on the given dataframe

        if (c == 'Income'): # if we the given column is the Income 

            if pd.isnull(checkpoint_2[sequence[4]][c][idx]): # if the given Household Key has a null value for Income column
            
                checkpoint_2[sequence[4]].at[idx, c] = 9999 # we replace it by the lowest value of income possible

            else:

                item = checkpoint_2[sequence[4]][c][idx] # we save our string list

                if ("K+" in item): # that means that we have the highest possible value of Income on the given Household Key

                    checkpoint_2[sequence[4]].at[idx, c] = 250000 # so we set as the only the higher value

                elif ("Und" in item): # that means that we have the lowest possible value of Income on the given Household Key

                    if len(item.split('-')) == 1: # if we only have this value

                        checkpoint_2[sequence[4]].at[idx, c] = 9999 # we replace it by the lowest value of income possible

                    else: # if we have other values besides the lower Income possible

                        lower_limit = 9999 # we set the lower limit by the lowest value of income possible

                        tmp = [] # we create an empty list

                        for s in item.split('-'): # we iterate over all values for this credit range
                            
                            tmp.append(s.split('Und')[0].split(',')[0]) # we append all numerical strings to it

                        tmp.sort() # we sort the list of numerical strings

                        upper_limit = float(tmp[-1]) * 1e3 # we choose the biggest value 

                        checkpoint_2[sequence[4]].at[idx, c] = (upper_limit + lower_limit)/2 # we replace the total value for the given Houlsehold Key by the mean

                else:

                        if len(item.split('-')) == 2: # if we only have two values for this range

                            lower_limit =  float(item.split('-')[0]) * 1e3 # lower Income limit

                            upper_limit = float(item.split('-')[1].split(',')[0]) * 1e3 # Upper Income limit

                            checkpoint_2[sequence[4]].at[idx, c] = (upper_limit + lower_limit)/2 # we replace the total value for the given Household Key by the mean

                        else: # we have more than two values available

                            tmp = [] # we create an empty list

                            for s in item.split('-'): # we iterate over all values for this income
                            
                                tmp.append(s.split('Und')[0].split(',')[0]) # we append all numerical strings to it

                            tmp.sort() # we sort the list of numerical strings

                            lower_limit = float(tmp[0]) * 1e3 # we choose the smaller value 
                                
                            upper_limit = float(tmp[-1]) * 1e3 # we choose the biggest value

                            checkpoint_2[sequence[4]].at[idx, c] = (upper_limit + lower_limit)/2 # we replace the total value for the given Houlsehold Key by the mean

checkpoint_2[sequence[4]]['Income'] = checkpoint_2[sequence[4]]['Income'].astype(float) # converting to float type

checkpoint_2[sequence[4]].rename(columns={'Income' : 'Annual Income'}, inplace=True) # renaming the colum

            6.6.2.3 - Education

In [113]:
for idx in tqdm_notebook(checkpoint_2[sequence[4]].index, desc='Process Progress'): # For each row
    for c in checkpoint_2[sequence[4]].columns: # For all columns on the given dataframe

        if (c == 'Education'): # if we the given column is the Education 

            if pd.isnull(checkpoint_2[sequence[4]][c][idx]): # if the given Household Key has a null value for Education column
            
                checkpoint_2[sequence[4]].at[idx, c] = 'Unknown' # we replace it by Unknow Education background
            
            else:

                item = checkpoint_2[sequence[4]][c][idx] # we save our string 

                if ("Unk" in item): # that means that we have an Unknown flag in the middle of it

                    if len(item.split()) == 1: # if we only have this value

                        pass # we pass it because it is already defined

                    else: # if we have other values besides the lower Income possible

                        checkpoint_2[sequence[4]].at[idx, c] = item.split('Unk')[0] # we replace the value by the string that is not Unknown

                else:

                        if (len(item.split()) > 2) and ("College" in item): # if we have more than two values for this range and college is one of them
                            if ('Completed' in item.split('College')): # we check if there is one member with a complete college degree

                                checkpoint_2[sequence[4]].at[idx, c] = 'Completed College' # we replace by the higgest

                            else:

                                checkpoint_2[sequence[4]].at[idx, c] = 'Some College' # we replace by the middle one

                        if (len(item.split()) > 2) and ("School" in item): # if we have more than two values for this range and school is one of them
                            if ('Graduated' in item.split('School')): # we check if there is one member with a complete school degree

                                checkpoint_2[sequence[4]].at[idx, c] = 'Graduated School' # we replace by the higgest one

                            else:

                                checkpoint_2[sequence[4]].at[idx, c] = 'High School' # we replace by the lower one


            6.6.2.4 - Member Status, Member Type, Renew Method, and Motorcycle Indicator

In [114]:
for idx in tqdm_notebook(checkpoint_2[sequence[4]].index, desc='Process Progress'): # For each row
    for c in checkpoint_2[sequence[4]].columns: # For all columns on the given dataframe

        if (c == 'Member Status'): # if we the given column is the Member Status 

            item = checkpoint_2[sequence[4]][c][idx] # we save our string 

            if item not in ['ACTIVE', 'PENDING', 'CANCELLED']: # if the value is not one of the possible ones

                if 'ACTIVE' in item: # if one of the possible values for the given Household Key is Active

                    checkpoint_2[sequence[4]].at[idx, c] = 'ACTIVE' # we replace it by an Active key

                else:
                    
                    checkpoint_2[sequence[4]].at[idx, c] = 'PENDING' # we replace it by an Pending key
                        
        elif (c == 'Plus Indicator Description'): # if the given column is the Member Type

            item = checkpoint_2[sequence[4]][c][idx] # we save our string 

            if pd.isnull(item):

                checkpoint_2[sequence[4]].at[idx, c] = 'Basic' # we replace it by an Basic Membership

            elif 'Premier' in item: # if one of the possible values for the given Household Key is Premier

                checkpoint_2[sequence[4]].at[idx, c] = 'Premier' # we replace it by the Pending key

            elif 'Plus' in item: # if one of the possible values for the given Household Key is Plus 

                checkpoint_2[sequence[4]].at[idx, c] = 'Plus' # we replace it by the Plus key

            else: # otherwise

                checkpoint_2[sequence[4]].at[idx, c] = 'Basic' # we replace it by the Basic key


        elif (c == 'Renew Method'): # if the given column is the Renew Method

            item = checkpoint_2[sequence[4]][c][idx] # we save our string 

            if 'AUTO' in item: # if one of the possible values for the given Household Key is Auto Renew

                checkpoint_2[sequence[4]].at[idx, c] = 'AUTO RENEW' # we replace it by the Auto Rener key

        elif (c == 'Motorcycle Indicator'): # if the given column is the Renew Method

            item = checkpoint_2[sequence[4]][c][idx]

            if pd.isnull(item) or ('N' in item): # if the given Household Key has a null value for Motorcycle Indicator column

                checkpoint_2[sequence[4]].at[idx, c] = 0 # we replace it by N Motorcycle Indicator

            elif 'Y' in item: # if one of the possible values for the given Household Key is Primary

                checkpoint_2[sequence[4]].at[idx, c] = 1 # we replace it by Y Motorcycle Indicator

checkpoint_2[sequence[4]]['Motorcycle Indicator'] = checkpoint_2[sequence[4]]['Motorcycle Indicator'].astype(int) # converting to iterger type
checkpoint_2[sequence[4]].rename(columns={'Plus Indicator Description' : 'Membership Type'}, inplace=True) # renaming the colum

            6.6.2.5 - Prob1 Code Description, SC Vehicle Manufacturer Name and SC Call Club Code Description

In [115]:
checkpoint_2[sequence[4]]['AAA Service Necessity Due Problem' + ' ' + str(2)] = '' # we create another column on the dataframe to store the other different value

checkpoint_2[sequence[4]]['Vehicle Manufacturer Name' + ' ' + str(2)] = '' # we create another column on the dataframe to store the other different value

checkpoint_2[sequence[4]]['AAA Club' + ' ' + str(2)] = '' # we create another column on the dataframe to store the other different value

In [116]:
for idx in tqdm_notebook(checkpoint_2[sequence[4]].index, desc='Process Progress'): # For each row
    for c in checkpoint_2[sequence[4]].columns: # For all columns on the given dataframe

        if (c == 'Prob1 Code Description'): # if we the given column is the Prob1 Code Description 

            item = checkpoint_2[sequence[4]][c][idx] # we save our string

            if pd.isnull(item): # if the value of out string is null

                checkpoint_2[sequence[4]].at[idx, c] = 'No record for the last three years' # we predict saying there is no record for the last three years

            if item not in df_original[c].unique().tolist(): # if our item is not on the original unique values list for this column

                out = "" # string to store the second value available

                for s in item: # work on character by character on our string

                    out = out + item[-1] # the second value building

                    item = item[:-1] # popping the last character on the string

                    if out[::-1] in df_original[c].unique().tolist(): # if the build of the second value is ready

                        checkpoint_2[sequence[4]].at[idx, 'AAA Service Necessity Due Problem 2'] = out[::-1] # we input the second value on the second column

                    if item in df_original[c].unique().tolist(): # the build of the first value is ready

                        checkpoint_2[sequence[4]].at[idx, c] = item # we input the first value on the original column

        elif (c == 'SC Vehicle Manufacturer Name'):

            item = checkpoint_2[sequence[4]][c][idx] # we save our string

            if pd.isnull(item): # if the value of out string is null

                checkpoint_2[sequence[4]].at[idx, c] = 'No record for the last three years' # we predict saying there is no record for the last three years

            if item not in df_original[c].unique().tolist(): # if our item is not on the original unique values list for this column

                out = "" # string to store the second value available

                for s in item: # work on character by character on our string

                    out = out + item[-1] # the second value building

                    item = item[:-1] # popping the last character on the string

                    if out[::-1] in df_original[c].unique().tolist(): # if the build of the second value is ready

                        checkpoint_2[sequence[4]].at[idx, 'Vehicle Manufacturer Name 2'] = out[::-1] # we input the second value on the second column
                    if item in df_original[c].unique().tolist(): # the build of the first value is ready

                        checkpoint_2[sequence[4]].at[idx, c] = item # we input the first value on the original column 

        elif (c == 'SC Call Club Code Description'):

            item = checkpoint_2[sequence[4]][c][idx] # we save our string

            if pd.isnull(item): # if the value of out string is null

                checkpoint_2[sequence[4]].at[idx, c] = 'No record for the last three years' # we predict saying there is no record for the last three years 

            if item not in df_original[c].unique().tolist(): # if our item is not on the original unique values list for this column

                out = "" # string to store the second value available

                for s in item: # work on character by character on our string

                    out = out + item[-1] # the second value building

                    item = item[:-1] # popping the last character on the string

                    if out[::-1] in df_original[c].unique().tolist(): # if the build of the second value is ready

                        checkpoint_2[sequence[4]].at[idx, 'AAA Club 2'] = out[::-1] # we input the second value on the second column
                    if item in df_original[c].unique().tolist(): # the build of the first value is ready

                        checkpoint_2[sequence[4]].at[idx, c] = item # we input the first value on the original column                       

checkpoint_2[sequence[4]].rename(columns={'Prob1 Code Description' : 'AAA Service Necessity Due Problem 1'}, inplace=True) # renaming the column
checkpoint_2[sequence[4]].rename(columns={'SC Vehicle Manufacturer Name' : 'Vehicle Manufacturer Name 1'}, inplace=True) # renaming the column
checkpoint_2[sequence[4]].rename(columns={'SC Call Club Code Description' : 'AAA Club 1'}, inplace=True) # renaming the column

        6.6.3 - Appending column to numerical one

In [117]:
for c in checkpoint_2[sequence[4]].columns: # For all columns on the given dataframe

    checkpoint_2[sequence[0]][c] = checkpoint_2[sequence[4]][c].values # concatenate the columns on the numerical dataframe

## 7 - Creating required columns for assignment

    7.1 - Number of Members per Household Key

        7.1.1 - Creating new Column

In [118]:
checkpoint_2[sequence[0]]['Number of Members per Household Key'] = 0 # creating ner column

checkpoint_2[sequence[0]]['Number of Members per Household Key'] = checkpoint_2[sequence[0]]['Number of Members per Household Key'].astype(int) # converting to iterger type

        7.1.2 - List of members per each Household Key on the original dataset

In [119]:
member_per_household = Counter(list(df_original['Household Key'].values)) # list of number of times that a given Household Key appears on the original dataframe

        7.1.3 - Filling Values

In [120]:
for idx in tqdm_notebook(checkpoint_2[sequence[0]].index, desc='Process Progress'): # For each row

    checkpoint_2[sequence[0]].at[idx, 'Number of Members per Household Key'] = member_per_household[idx] # replace the value by the number of members


    7.2 - Frequency of use of Roadside Services and if Household have used the Roadside Services

        7.2.1 - Creating new Column

In [121]:
checkpoint_2[sequence[0]]['Frequency of use of Roadside Services'] = 0 # creating ner column

checkpoint_2[sequence[0]]['Frequency of use of Roadside Services'] = checkpoint_2[sequence[0]]['Frequency of use of Roadside Services'].astype(int) # converting to iterger type

checkpoint_2[sequence[0]]['Household have used the Roadside Services'] = 0 # creating ner column

checkpoint_2[sequence[0]]['Household have used the Roadside Services'] = checkpoint_2[sequence[0]]['Household have used the Roadside Services'].astype(int) # converting to iterger type

        7.2.2 - Filling values

In [122]:
for idx in tqdm_notebook(checkpoint_2[sequence[0]].index, desc='Process Progress'): # For each row

    checkpoint_2[sequence[0]].at[idx, 'Frequency of use of Roadside Services'] = checkpoint_2[sequence[0]]['Number of Roadside calss for Year 1'][idx] +  checkpoint_2[sequence[0]]['Number of Roadside calss for Year 2'][idx] + checkpoint_2[sequence[0]]['Number of Roadside calss for Year 3'][idx] # filling the column with the total number of usage of roadside services


    if (checkpoint_2[sequence[0]]['Number of Roadside calss for Year 1'][idx] != 0) or (checkpoint_2[sequence[0]]['Number of Roadside calss for Year 2'][idx] != 0) or (checkpoint_2[sequence[0]]['Number of Roadside calss for Year 3'][idx] != 0): # if the given Household Key has used any roadside service on the last three years

        checkpoint_2[sequence[0]].at[idx, 'Household have used the Roadside Services'] = 1

## 8 - Final Dataframe

    8.1 - Creating Dataframe

In [123]:
df_final = checkpoint_2[sequence[0]].copy() # copying final post-processed dataframe

    8.2 - Checking for Null data 

In [124]:
# plt.figure(figsize=(10,5))

# sns.barplot(x=df_final.isnull().sum().index, y=df_final.isnull().sum())
# plt.xticks(rotation='90')
# plt.xlabel('Variables', fontsize=10)
# plt.ylabel('Percentage of missing values', fontsize=10)
# plt.title('Percentage of missing values for the Final Dataframe', fontsize=15)

    8.3 - Checking Data Types

In [125]:
df_final.head()

,Length Of Residence,Number of Roadside calss for Year 1,Number of Roadside calss for Year 2,Number of Roadside calss for Year 3,Roadside Member Cost for Year 1,Roadside Member Cost for Year 2,Roadside Member Cost for Year 3,Member Tenure Years,Years to Cancelled Status,Basic Cost,Calculated Tow Miles,Plus Cost,Premier Cost,Total Cost,Total Number of Roadside Calls,Total Roadside Member Cost,Email Available,Age,Years from most recent Membership Expiration,Years from Join Date,Purchased Product 1,Purchased Product 2,Purchased Product 3,Purchased Product 4,Purchased Product 5,Purchased Product 6,Purchased Product 7,Purchased Product 8,Mail Responder,Home Owner,Opt-Out - Publication,Cash Call,Fleet Indicator,Total Number of Purchased Products,Household Purchased Product,Email Correspondent,Home Renter,No Home Ownership Info,City,Number of Children,Annual Income,Education,Annual Credit Possible,Member Status,Renew Method,Motorcycle Indicator,Membership Type,AAA Service Necessity Due Problem 1,AAA Club 1,Vehicle Manufacturer Name 1,AAA Service Necessity Due Problem 2,Vehicle Manufacturer Name 2,AAA Club 2,Number of Members per Household Key,Frequency of use of Roadside Services,Household have used the Roadside Services
Household Key,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8750,90.558333,8,4,7,4623.928571,2522.142857,5472.0,289,131,1250.357143,80.142857,265.714286,0.0,1516.071429,19,12618.071429,1,74,2,30,0,1,0,0,0,0,0,0,1,1,1,0,0,1,1,1,0,0,WEST WARWICK,1,34500.0,Some College,624500.0,ACTIVE,AUTO RENEW,0,Basic,Battery Service,AAA Northeast,TOYOTA,,,,18,19,1
9690,140.000000,0,7,15,0.000000,1470.833333,15059.5,353,242,3072.500000,22.500000,127.500000,0.0,4347.500000,22,16530.333333,1,76,0,36,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,1,0,0,WARWICK,0,54500.0,Unknown,774500.0,ACTIVE,AUTO RENEW,0,Basic,Flat Tire,AAA Northeast,SUBARU,,,,6,22,1
33380,50.000000,0,0,0,0.000000,0.000000,0.0,373,260,0.000000,0.000000,0.000000,0.0,0.000000,0,0.000000,0,90,11,38,0,0,0,0,0,0,1,0,1,1,1,0,0,1,1,1,0,0,CENTRAL FALLS,0,54500.0,Some College,674500.0,ACTIVE,NOTICE,0,Basic,Lockout,AAA Northeast,INFINITI,,,,3,0,0
87180,150.000000,0,0,0,0.000000,0.000000,0.0,100,2,0.000000,0.000000,0.000000,0.0,0.000000,0,0.000000,1,68,3,11,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,1,0,0,WARWICK,1,124500.0,Unknown,674500.0,PENDING,NOTICE,0,Basic,Jump Start,AAA Northeast,VOLKSWAGEN,,,,2,0,0
115240,121.000000,3,22,10,1961.666667,12205.833333,530.0,363,93,5885.000000,12.000000,0.000000,0.0,5885.000000,35,14697.500000,1,50,3,37,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,1,0,0,BARRINGTON,4,124500.0,Unknown,800000.0,ACTIVE,NOTICE,0,Basic,Extrication,AAA Northeast,LEXUS,Flat Tire,,,7,35,1


    We still need to convert the string variables

    8.4 - Categoriacal One Hot Enconding

In [126]:
for c in tqdm_notebook(df_final.columns, desc='Process Progress'): # For all columns on the given dataframe

    if df_final.dtypes[c] == 'O': # wecheck if the given column is an object type

        df_final = pd.get_dummies(df_final, columns=[c], dtype=int) # we replace it by the binary representaion

## 9 - Purchase Probability Prediction

    9.1 - Saving our third checkpoint 

In [127]:
checkpoint_3 = df_final.copy() # copying final checkpoint

    9.2 - Taking a look at our Target Variable

In [128]:
from scipy import stats # for some statistics
#from scipy.stats import norm, skew 

In [129]:
(mu, sigma) = stats.norm.fit(checkpoint_3['Household Purchased Product'] ) # calculating mean and standard deviation of target variable

In [130]:
# sns.distplot(checkpoint_3['Household Purchased Product']); # distribution plot with normalized fitting

# plt.legend('Normal distribution')
# plt.ylabel('Frequency')
# plt.title('Household Purchased Product distribution -- Mu:' + ' ' + str(mu) + ' ' + 'Sigma:' + ' ' + str(sigma) )

    As a binary variable we wont need to standardize it

    9.3 - Checking our data correlation

In [131]:
corrmat = abs(checkpoint_3.corr()) # dataframe correlation

In [132]:
#plt.figure(figsize=(20,20))
#sns.heatmap(corrmat, square=True, robust=True); # matrix correlation

    9.4 - Feature Selection

        9.4.1 - Pearson Correlation (Filtering Metod)

    Let's target to our objective. The prediction of the Percentage of a given Household buy a AAA product

In [133]:
k = 10 #number of variables for heatmap

cols = corrmat.nlargest(k, 'Household Purchased Product')['Household Purchased Product'].index # The tenth greater correlation columns

focus_corr = abs(np.corrcoef(checkpoint_3[cols].values.T)) # correlation calculated considering only the tenth more correlated columns

#plt.figure(figsize=(15,10))

#hm = sns.heatmap(focus_corr, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values) # correlation matrix

#plt.show()

    We have narrow down to 10 features. However, for linear regression, we still need to check intra-dependencies among those features. In here, we will consider a set of dependent variables when they have a correlation magnitude higher than 55%.

In [134]:
cols_greater = cols[1:-2] # removing two last columns only to remain the higher correlations values and remove the target variable for this view

focus_corr_greater = abs(np.corrcoef(checkpoint_3[cols_greater].values.T)) # correlation calculated considering only the tenth more correlated columns

#plt.figure(figsize=(15,10))

#hm = sns.heatmap(focus_corr_greater, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols_greater.values, xticklabels=cols_greater.values) # correlation matrix

#plt.show()

    As we can see, the columns Total Number of Purchased Product and Purchased Product 7 are highly correlated. Therefore, we only need to maintain one of them. In this case, we will maintain the Total Number of Purchased Product because it has a higher correlation score with the target feature than the other.

In [135]:
pearson_features = cols_greater.drop('Purchased Product 7').tolist() # saving our features selecter using the filtering method (Pearson Correlation)

        9.4.2 - Recursive Feature Elimination

            9.4.2.1 - Sklearn libraries for analysis

In [136]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

            9.4.2.2 - Create the the label data, expected output using our checkpoint3 and converting it to array

In [137]:
entry = checkpoint_3.drop(columns='Household Purchased Product').values # converting our features to be analyzed by the RFE method to array type

target = checkpoint_3['Household Purchased Product'].values # converting our target to array type

In [138]:
model = LogisticRegression(solver='lbfgs', n_jobs= 9, penalty='l2') # lbfgs solver is recommended for large datasets and we choose l1 norm for a better convergency

            9.4.2.4 - RFE experiment for multiples number of possible features

In [139]:
import numpy.matlib as matlib # for data augmentation procedure

number_possible_features = np.arange(3, 10) # list of number of possible features
           
features_list_score = [] # list to track calculated scores

selected_features = [] # list to track selected features

for n in tqdm_notebook(range(len(number_possible_features)), desc='RFE Optimum Number of Features'): # for all possibel number of optimum features

        rfe = RFE(model, number_possible_features[n]) # defining RFE experiment

        rfe.fit(MinMaxScaler().fit_transform(StandardScaler().fit_transform(entry)), target); # fitting

        temp = pd.Series(rfe.support_,  index = list(checkpoint_3.drop(columns='Household Purchased Product').columns)) # mask for RFE features

        tmp_selected_features = temp[temp==True].index.tolist() # selecting columns where the mask is True

        selected_features.append(tmp_selected_features) # appending to our list

        x_train, x_test, y_train, y_test = train_test_split(MinMaxScaler().fit_transform(StandardScaler().fit_transform(checkpoint_3[tmp_selected_features])), target, test_size = 0.7, random_state = 42, shuffle=True) # splitting dataset as train and test


        x2 = np.matlib.repmat(x_train, 10, 1) # as we have an imbalanced sample of classes, we perform a data augmentation procedure to avoid miss prediction

        y2 = np.matlib.repmat(y_train, 10, 1).ravel() # as we have an imbalanced sample of classes, we perform a data augmentation procedure to avoid miss prediction

        model.fit(x2, y2) # fitting the model

        tmp_score = model.score(x_test, y_test) # scoring the model 

        features_list_score.append(tmp_score) # appending to the score list
        

In [140]:
for i in range(len(selected_features)):

    print('Selected features for K =' + ' ' + str(i+3), '\n')

    print(selected_features[i], '\n')

    print('Corresponding Score:' + ' ' + str(features_list_score[i]))

Selected features for K = 3 

['Purchased Product 7', 'Purchased Product 8', 'Total Number of Purchased Products'] 

Corresponding Score: 1.0
Selected features for K = 4 

['Purchased Product 2', 'Purchased Product 7', 'Purchased Product 8', 'Total Number of Purchased Products'] 

Corresponding Score: 1.0
Selected features for K = 5 

['Purchased Product 1', 'Purchased Product 2', 'Purchased Product 7', 'Purchased Product 8', 'Total Number of Purchased Products'] 

Corresponding Score: 1.0
Selected features for K = 6 

['Purchased Product 1', 'Purchased Product 2', 'Purchased Product 5', 'Purchased Product 7', 'Purchased Product 8', 'Total Number of Purchased Products'] 

Corresponding Score: 0.9979072200906871
Selected features for K = 7 

['Purchased Product 1', 'Purchased Product 2', 'Purchased Product 3', 'Purchased Product 5', 'Purchased Product 7', 'Purchased Product 8', 'Total Number of Purchased Products'] 

Corresponding Score: 0.9996512033484478
Selected features for K = 8 



    We have seen that for K till 5 we maintain a score equal 1. In this sense we choose the biggest K to use the most of the available features

In [141]:
rfe_features = selected_features[2] # choosing RFE selected features

        9.5.1 - For Features Selected using the Filtering Method

    9.5 - Prediction of Purchased Probability using select features

             9.5.1.1 - Performing Prediction

In [142]:
model = LogisticRegression(solver='lbfgs', n_jobs= 9, penalty='l2') # lbfgs solver is recommended for large datasets and we choose l2 norm for a better convergency

x_train, x_test, y_train, y_test = train_test_split(MinMaxScaler().fit_transform(StandardScaler().fit_transform(checkpoint_3[pearson_features])), target, test_size = 0.7, random_state = 42, shuffle=True) # splitting dataset using features selected for the filtering method

x2 = np.matlib.repmat(x_train, 10, 1) # as we have an imbalanced sample of classes, we perform a data augmentation procedure to avoid miss prediction

y2 = np.matlib.repmat(y_train, 10, 1).ravel() # as we have an imbalanced sample of classes, we perform a data augmentation procedure to avoid miss prediction

model.fit(x2, y2) # fitting the model

model.score(x_test, y_test) # scoring the model 

1.0

            9.5.1.2 - Analyzing Result 

In [143]:
pearson_feat_pred = model.predict(MinMaxScaler().fit_transform(StandardScaler().fit_transform(checkpoint_3[pearson_features]))) # predicting the target variable using the entire input

pearson_feat_pred_proba = model.predict_proba(MinMaxScaler().fit_transform(StandardScaler().fit_transform(checkpoint_3[pearson_features]))) # predicting the probability of belonging to a given class of the target variable using the entire input

            9.5.1.3 - Analyzing Metrics 

In [144]:
calss_ids = ['Household did not Purchased any Product', 'Household did Purchased some Product'] # classes names

print(classification_report(target, pearson_feat_pred, target_names=calss_ids)) # classification report

precision    recall  f1-score   support

Household did not Purchased any Product       1.00      1.00      1.00      2594
   Household did Purchased some Product       1.00      1.00      1.00      1501

                               accuracy                           1.00      4095
                              macro avg       1.00      1.00      1.00      4095
                           weighted avg       1.00      1.00      1.00      4095



        9.5.2 - For Features Selected using the RFE Method

            9.5.2.1 - Performing Prediction

In [145]:
model = LogisticRegression(solver='lbfgs', n_jobs= 9, penalty='l2') # lbfgs solver is recommended for large datasets and we choose l2 norm for a better convergency

x_train, x_test, y_train, y_test = train_test_split(MinMaxScaler().fit_transform(StandardScaler().fit_transform(checkpoint_3[rfe_features])), target, test_size = 0.7, random_state = 42, shuffle=True) # splitting dataset using features selected for the filtering method

x2 = np.matlib.repmat(x_train, 10, 1) # as we have an imbalanced sample of classes, we perform a data augmentation procedure to avoid miss prediction

y2 = np.matlib.repmat(y_train, 10, 1).ravel() # as we have an imbalanced sample of classes, we perform a data augmentation procedure to avoid miss prediction

model.fit(x2, y2) # fitting the model

model.score(x_test, y_test) # scoring the model 

1.0

            9.5.2.2 - Analyzing Result 

In [146]:
rfe_feat_pred = model.predict(MinMaxScaler().fit_transform(StandardScaler().fit_transform(checkpoint_3[rfe_features]))) # predicting the target variable using the entire input

rfe_feat_pred_proba = model.predict_proba(MinMaxScaler().fit_transform(StandardScaler().fit_transform(checkpoint_3[rfe_features]))) # predicting the probability of belonging to a given class of the target variable using the entire input

            9.5.2.3 - Analyzing Metrics 

In [147]:
print(classification_report(target, rfe_feat_pred, target_names=calss_ids)) # classification report

precision    recall  f1-score   support

Household did not Purchased any Product       1.00      1.00      1.00      2594
   Household did Purchased some Product       1.00      1.00      1.00      1501

                               accuracy                           1.00      4095
                              macro avg       1.00      1.00      1.00      4095
                           weighted avg       1.00      1.00      1.00      4095



    Both approaches present solid results, but we still need to choose one of them. Lets see the most accurate probability among them.

In [148]:
class_repo = pd.DataFrame.from_dict(classification_report(target, rfe_feat_pred, target_names=calss_ids, output_dict=True)) # passing classification report to dataframe

In [149]:
class_repo.rename(index={'precision': 'Precision', 'recall': 'Recall', 'f1-score': 'F1-Score'}, inplace=True) # renaming columns

In [150]:
#plt.figure(figsize=(10,5))

#sns.barplot(x=class_repo['Household did Purchased some Product'][0:-1].index, y=class_repo['Household did Purchased some Product'][0:-1])
#plt.xlabel('Metrics', fontsize=12, weight='bold')
#plt.ylabel('Normalized Percentage', fontsize=12, weight='bold')
#plt.title('Model Metrics for Purchase Product Probability', fontsize=15, weight='bold')
#plt.savefig('model_metrics_purch.png', dpi=200, bbox_inches='tight')
#plt.show()

    9.6 - Approach Choice and Creation of the new column

In [151]:
abs((pearson_feat_pred_proba - 1)[:, 1]).sum() # total distance between the binary correct label

2595.190840728762

In [152]:
abs((rfe_feat_pred_proba - 1)[:, 1]).sum()  # total distance between the binary correct label

2592.359024894099

    As we can see, the RFE method provided a more accurate prediction than the Filtering Method. Therefore, we will use the RFE probability prediction to generate our new column

In [153]:
checkpoint_3['Household Purchase Product Probability'] = rfe_feat_pred_proba[:, 1] # creating new column

## 10 - Roadside Usage Probability Prediction

    10.1 - Taking a look at our Target Variable

In [154]:
(mu, sigma) = stats.norm.fit(checkpoint_3['Household have used the Roadside Services'] ) # calculating mean and standard deviation of target variable


In [155]:
# sns.distplot(checkpoint_3['Household have used the Roadside Services']); # distribution plot with normalized fitting

# plt.legend('Normal distribution')
# plt.ylabel('Frequency')
# plt.title('Household have used the Roadside Services distribution -- Mu:' + ' ' + str(mu) + ' ' + 'Sigma:' + ' ' + str(sigma) )

    In this case, the variable presents a even more unbalanced distribution than the previous one

    10.2 - Feature Selection

        10.2.1 - Pearson Correlation (Filtering Metod)

In [156]:
k = 10 #number of variables for heatmap

cols = corrmat.nlargest(k, 'Household have used the Roadside Services')['Household have used the Roadside Services'].index # The tenth greater correlation columns

focus_corr = abs(np.corrcoef(checkpoint_3[cols].values.T)) # correlation calculated considering only the tenth more correlated columns

#plt.figure(figsize=(15,10))

#hm = sns.heatmap(focus_corr, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values) # correlation matrix

#plt.show()

                We have narrow down to 10 features. However, for linear regression, we still need to check intra-dependencies among those features. In here, we will consider all available variables in a way they all make sense to be related to the target variable.


In [157]:
cols_greater = cols[1::]

focus_corr_greater = abs(np.corrcoef(checkpoint_3[cols_greater].values.T)) # correlation calculated considering only the tenth more correlated columns

#plt.figure(figsize=(15,10))

#hm = sns.heatmap(focus_corr_greater, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols_greater.values, xticklabels=cols_greater.values) # correlation matrix

#plt.show()

    As we can see, the columns Total Number of Roadside Calss, Frequency of use of Roadside Services are the same. Therefore, we only need to maintain one of them. In this case, we will maintain the first one.

In [158]:
pearson_features = cols_greater.drop('Frequency of use of Roadside Services').tolist() # saving our features selecter using the filtering method (Pearson Correlation)


        10.2.2 - Recursive Feature Elimination


            10.2.2.1 - Create the the label data, expected output using our checkpoint3 and converting it to array

In [159]:
entry = checkpoint_3.drop(columns='Household have used the Roadside Services').values # converting our features to be analyzed by the RFE method to array type

target = checkpoint_3['Household have used the Roadside Services'].values # converting our target to array type


            10.2.2.2 - RFE experiment for multiples number of possible features

In [160]:
import numpy.matlib as matlib # for data augmentation procedure

number_possible_features = np.arange(3, 10) # list of number of possible features
           
features_list_score = [] # list to track calculated scores

selected_features = [] # list to track selected features

for n in tqdm_notebook(range(len(number_possible_features)), desc='RFE Optimum Number of Features'):

        rfe = RFE(model, number_possible_features[n]) # defining RFE experiment

        rfe.fit(MinMaxScaler().fit_transform(StandardScaler().fit_transform(entry)), target); # fitting

        temp = pd.Series(rfe.support_,  index = list(checkpoint_3.drop(columns='Household have used the Roadside Services').columns)) # selecting RFE columns features

        tmp_selected_features = temp[temp==True].index.tolist()

        selected_features.append(tmp_selected_features) # appending to our list

        x_train, x_test, y_train, y_test = train_test_split(MinMaxScaler().fit_transform(checkpoint_3[tmp_selected_features]), target, test_size = 0.7, random_state = 42, shuffle=True)


        x2 = np.matlib.repmat(x_train, 10, 1) # as we have an imbalanced sample of classes, we perform a data augmentation procedure to avoid miss prediction

        y2 = np.matlib.repmat(y_train, 10, 1).ravel() # as we have an imbalanced sample of classes, we perform a data augmentation procedure to avoid miss prediction

        model.fit(x2, y2) # fitting the model

        tmp_score = model.score(x_test, y_test) # scoring the model 

        features_list_score.append(tmp_score) # appending to the score list

In [161]:
for i in range(len(selected_features)):

    print('Selected features for K =' + ' ' + str(i+3), '\n')

    print(selected_features[i], '\n')

    print('Corresponding Score:' + ' ' + str(features_list_score[i]))

Selected features for K = 3 

['Number of Roadside calss for Year 2', 'Total Number of Roadside Calls', 'Frequency of use of Roadside Services'] 

Corresponding Score: 0.9794209975584235
Selected features for K = 4 

['Number of Roadside calss for Year 2', 'Number of Roadside calss for Year 3', 'Total Number of Roadside Calls', 'Frequency of use of Roadside Services'] 

Corresponding Score: 0.9853505406348099
Selected features for K = 5 

['Number of Roadside calss for Year 1', 'Number of Roadside calss for Year 2', 'Number of Roadside calss for Year 3', 'Total Number of Roadside Calls', 'Frequency of use of Roadside Services'] 

Corresponding Score: 0.9794209975584235
Selected features for K = 6 

['Number of Roadside calss for Year 1', 'Number of Roadside calss for Year 2', 'Number of Roadside calss for Year 3', 'Total Number of Roadside Calls', 'Total Roadside Member Cost', 'Frequency of use of Roadside Services'] 

Corresponding Score: 0.9825601674223927
Selected features for K = 7

In [162]:
idx = features_list_score.index(max(features_list_score)) # index of max value on the scores list

rfe_features = selected_features[idx] # choosing RFE selected features

    10.3 - Prediction of Roadside Services Usage Probability using select features 

        10.3.1 - For Features Selected using the Filtering Method

            10.3.1.1 - Performing Prediction

In [163]:
x_train, x_test, y_train, y_test = train_test_split(MinMaxScaler().fit_transform(checkpoint_3[pearson_features]), target, test_size = 0.7, random_state = 42, shuffle=True) # splitting dataset using features selected for the filtering method

x2 = np.matlib.repmat(x_train, 10, 1) # as we have an imbalanced sample of classes, we perform a data augmentation procedure to avoid miss prediction

y2 = np.matlib.repmat(y_train, 10, 1).ravel() # as we have an imbalanced sample of classes, we perform a data augmentation procedure to avoid miss prediction

model.fit(x2, y2) # fitting the model

model.score(x_test, y_test) # scoring the model 

0.978374607603767

            10.3.1.2 - Analyzing Result

In [164]:
pearson_feat_pred = model.predict(MinMaxScaler().fit_transform(checkpoint_3[pearson_features])) # predicting the target variable using the entire input

pearson_feat_pred_proba = model.predict_proba(MinMaxScaler().fit_transform(checkpoint_3[pearson_features])) # predicting the probability of belonging to a given class of the target variable using the entire input

            10.3.1.3 - Analyzing Metrics 

In [165]:
calss_ids = ['Household did not use Roadside Services', 'Household did use Roadside Services'] # classes names


print(classification_report(target, pearson_feat_pred, target_names=calss_ids)) # classification report

precision    recall  f1-score   support

Household did not use Roadside Services       0.93      0.99      0.96      1152
    Household did use Roadside Services       1.00      0.97      0.98      2943

                               accuracy                           0.98      4095
                              macro avg       0.96      0.98      0.97      4095
                           weighted avg       0.98      0.98      0.98      4095



            10.3.2 - For Features Selected using the RFE Method

                10.3.2.1 - Performing Prediction

In [166]:
x_train, x_test, y_train, y_test = train_test_split(MinMaxScaler().fit_transform(checkpoint_3[rfe_features]), target, test_size = 0.7, random_state = 42, shuffle=True) # splitting dataset using features selected for the filtering method

x2 = np.matlib.repmat(x_train, 10, 1) # as we have an imbalanced sample of classes, we perform a data augmentation procedure to avoid miss prediction

y2 = np.matlib.repmat(y_train, 10, 1).ravel() # as we have an imbalanced sample of classes, we perform a data augmentation procedure to avoid miss prediction

model.fit(x2, y2) # fitting the model

model.score(x_test, y_test) # scoring the model 

0.9853505406348099

                10.3.2.2 - Analyzing Result

In [167]:
rfe_feat_pred = model.predict(MinMaxScaler().fit_transform(checkpoint_3[rfe_features])) # predicting the target variable using the entire input

rfe_feat_pred_proba = model.predict_proba(MinMaxScaler().fit_transform(checkpoint_3[rfe_features])) # predicting the probability of belonging to a given class of the target variable using the entire input

                10.3.1.3 - Analyzing Metrics 

In [168]:
print(classification_report(target, rfe_feat_pred, target_names=calss_ids)) # classification report

precision    recall  f1-score   support

Household did not use Roadside Services       0.94      1.00      0.97      1152
    Household did use Roadside Services       1.00      0.98      0.99      2943

                               accuracy                           0.98      4095
                              macro avg       0.97      0.99      0.98      4095
                           weighted avg       0.98      0.98      0.98      4095



    10.4 - Approach Choice and Creation of the new column

        In this case, the features selected by the RFE method had better metrics than the filtering method. So, we will use the probability prediction of this from this method

In [169]:
checkpoint_3['Household Roadside Usage Probability'] = rfe_feat_pred_proba[:, 1] # creating new column

## 11 - Clustering By Household Types

    11.1 - Last Checkpoint

In [170]:
checkpoint_4 = checkpoint_3.copy() # creating new dataframe

    11.2 - Scaling our final checkpoint to K-Means procedure

In [171]:
checkpoint_4 = StandardScaler().fit_transform(checkpoint_4) # standardize scaler    

checkpoint_4 = MinMaxScaler().fit_transform(checkpoint_4) # normalize scaler

checkpoint_4 = pd.DataFrame(checkpoint_4, index=checkpoint_3.index, columns=checkpoint_3.columns) # converting back to dataframe type

    11.3 - K-Means Clustering and Hyperparameter Tunning

        11.3.1 - Importing Library

In [172]:
from sklearn.cluster import KMeans 

        11.3.2 - Elbow Method of Analysis

In [173]:
wcss = [] # Within cluster sum of squares to analyze k-means performance

k_number_clusters = np.arange(1, 11) # define number of clusters to test

for k in tqdm_notebook(k_number_clusters, desc='K-Means Hyperparameter Tunning'): 

    kmeans = KMeans(n_clusters=k, init="k-means++", random_state=42, max_iter=500, n_jobs=9) # k-means model definition

    kmeans.fit(checkpoint_4) # fitting to our dataframe

    wcss.append(kmeans.inertia_) # appending intertia value to our list for further evaluation

In [174]:
# plt.figure(figsize=(12,6))    
# plt.plot(k_number_clusters, wcss, linewidth=2, color="red", marker ="8")
# plt.xlabel("K Clusters Value", fontsize=12, weight='bold')
# plt.xticks(np.arange(1,11,1))
# plt.ylabel("Sum of Squared Distances for each closest Cluster Center", fontsize=12, weight='bold')
# plt.title('K-Means Elbow Plot Evaluation Method', fontsize=15, weight='bold')
# plt.savefig('elbow_plot.png', dpi=200, bbox_inches='tight')
# plt.show()

    In here, we will consider that the optimum number o K cluster, the elbow in our plot above, is 4

    11.4 - Predict and Append Clusters as a new column on previous unscaled dataframe

        11.4.1 - K-Means Clustering Definition and Fitting

In [175]:
optimum_clustering = KMeans(n_clusters=4, init="k-means++", random_state=42, max_iter=500, n_jobs=9) # optimum k-means

optimum_clustering.fit_predict(checkpoint_4) # fit and predicitions

array([3, 1, 3, ..., 1, 0, 1])

        11.4.2 - Creating Classes Column on our new dataframe made from the previous one

In [176]:
classes_frame = checkpoint_2[sequence[0]].copy() # final checkpoint

In [177]:
classes_frame['Household Group Label'] = optimum_clustering.labels_ # pushing labels to final dataframe

classes_frame['Household Purchase Product Probability'] = checkpoint_3['Household Purchase Product Probability'].values # appending column

classes_frame['Household Roadside Usage Probability'] = checkpoint_3['Household Roadside Usage Probability'].values # appending column

classes_frame['Membership Type_Basic'] = checkpoint_4['Membership Type_Basic'].values # appending column

classes_frame['Membership Type_Plus'] = checkpoint_4['Membership Type_Plus'].values # appending column

classes_frame['Membership Type_Premier'] = checkpoint_4['Membership Type_Premier'].values # appending column

## 12 - Result Evaluation

In [179]:
class_0 = classes_frame.loc[classes_frame['Household Group Label'] == 0]

class_1 = classes_frame.loc[classes_frame['Household Group Label'] == 1]

class_2 = classes_frame.loc[classes_frame['Household Group Label'] == 2]

class_3 = classes_frame.loc[classes_frame['Household Group Label'] == 3]

    12.1 -  Class 0

        12.1.1 - Glance

In [180]:
class_0.head()

,Length Of Residence,Number of Roadside calss for Year 1,Number of Roadside calss for Year 2,Number of Roadside calss for Year 3,Roadside Member Cost for Year 1,Roadside Member Cost for Year 2,Roadside Member Cost for Year 3,Member Tenure Years,Years to Cancelled Status,Basic Cost,Calculated Tow Miles,Plus Cost,Premier Cost,Total Cost,Total Number of Roadside Calls,Total Roadside Member Cost,Email Available,Age,Years from most recent Membership Expiration,Years from Join Date,Purchased Product 1,Purchased Product 2,Purchased Product 3,Purchased Product 4,Purchased Product 5,Purchased Product 6,Purchased Product 7,Purchased Product 8,Mail Responder,Home Owner,Opt-Out - Publication,Cash Call,Fleet Indicator,Total Number of Purchased Products,Household Purchased Product,Email Correspondent,Home Renter,No Home Ownership Info,City,Number of Children,Annual Income,Education,Annual Credit Possible,Member Status,Renew Method,Motorcycle Indicator,Membership Type,AAA Service Necessity Due Problem 1,AAA Club 1,Vehicle Manufacturer Name 1,AAA Service Necessity Due Problem 2,Vehicle Manufacturer Name 2,AAA Club 2,Number of Members per Household Key,Frequency of use of Roadside Services,Household have used the Roadside Services,Household Group Label,Household Purchase Product Probability,Household Roadside Usage Probability,Membership Type_Basic,Membership Type_Plus,Membership Type_Premier
Household Key,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
197470,150.000000,0,0,0,0.000000,0.000000,0.000000,460,242,0.000000,0.000000,0.0,0.0,0.000000,0,0.000000,0,95,0,47,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,1,CRANSTON,0,9999.0,Unknown,499999.0,ACTIVE,AUTO RENEW,1,Basic,Battery Service,AAA Northeast,CHEVROLET,,,,1,0,0,0,0.010777,0.185457,1.0,0.0,0.0
204690,150.000000,37,9,9,2172.923077,5432.307692,489.230769,292,52,2654.583333,16.666667,0.0,0.0,2654.583333,55,8094.461538,0,77,0,30,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,NEWINGTON,0,9999.0,Unknown,499999.0,CANCELLED,NOTICE,0,Basic,Jump Start,AAA Northeast,HYUNDAI,,,,13,55,1,0,0.010777,1.000000,1.0,0.0,0.0
495780,150.000000,0,0,0,0.000000,0.000000,0.000000,210,242,150.000000,0.000000,0.0,0.0,150.000000,0,0.000000,1,76,0,22,0,0,0,0,0,0,1,1,1,0,1,0,0,2,1,1,0,1,COVENTRY,0,9999.0,Unknown,499999.0,ACTIVE,AUTO RENEW,0,Premier,Tow,AAA Northeast,CHRYSLER,,GMC,,3,0,0,0,1.000000,0.185457,0.0,0.0,1.0
552950,50.000000,0,0,0,0.000000,0.000000,0.000000,690,242,0.000000,0.000000,0.0,0.0,0.000000,0,0.000000,0,73,0,70,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,1,WARWICK,0,9999.0,Unknown,499999.0,ACTIVE,NOTICE,1,Basic,Battery Service,AAA Northeast,HYUNDAI,,,,1,0,0,0,0.010777,0.185457,1.0,0.0,0.0
1753100,105.737864,10,10,30,530.000000,530.000000,13575.000000,480,440,530.000000,0.000000,0.0,0.0,530.000000,50,14635.000000,0,74,4,49,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,1,NARRAGANSETT,0,9999.0,Unknown,499999.0,ACTIVE,NOTICE,0,Basic,Battery Service,AAA Northeast,HONDA,,TOYOTA,,1,50,1,0,0.010777,1.000000,1.0,0.0,0.0


        12.1.2 - General Description of Numerical Data

In [181]:
class_0.describe()

,Length Of Residence,Number of Roadside calss for Year 1,Number of Roadside calss for Year 2,Number of Roadside calss for Year 3,Roadside Member Cost for Year 1,Roadside Member Cost for Year 2,Roadside Member Cost for Year 3,Member Tenure Years,Years to Cancelled Status,Basic Cost,Calculated Tow Miles,Plus Cost,Premier Cost,Total Cost,Total Number of Roadside Calls,Total Roadside Member Cost,Email Available,Age,Years from most recent Membership Expiration,Years from Join Date,Purchased Product 1,Purchased Product 2,Purchased Product 3,Purchased Product 4,Purchased Product 5,Purchased Product 6,Purchased Product 7,Purchased Product 8,Mail Responder,Home Owner,Opt-Out - Publication,Cash Call,Fleet Indicator,Total Number of Purchased Products,Household Purchased Product,Email Correspondent,Home Renter,No Home Ownership Info,Number of Children,Annual Income,Annual Credit Possible,Motorcycle Indicator,Number of Members per Household Key,Frequency of use of Roadside Services,Household have used the Roadside Services,Household Group Label,Household Purchase Product Probability,Household Roadside Usage Probability,Membership Type_Basic,Membership Type_Plus,Membership Type_Premier
count,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.00000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.0,725.000000,725.000000,725.000000,725.0,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.0,725.000000,725.000000,725.000000,725.000000,725.000000
mean,116.613009,3.223448,5.834483,6.775172,753.859301,1547.240273,1042.144659,369.442759,246.14069,834.174257,18.663434,31.753410,0.082759,918.134890,15.833103,3343.244232,0.655172,74.532414,2.437241,37.948966,0.026207,0.066207,0.004138,0.001379,0.023448,0.0,0.172414,0.078621,0.590345,0.0,0.990345,0.001379,0.667586,0.372414,0.297931,0.990345,0.028966,0.971034,0.027586,14001.157241,512143.200000,0.169655,4.511724,15.833103,0.729655,0.0,0.301684,0.733699,0.554483,0.415172,0.030345
std,44.821608,6.702416,8.502205,9.890865,2222.421772,3623.012462,2989.496585,133.555143,135.76802,1135.087916,56.154625,181.842227,2.228344,1371.009691,17.935304,5807.502827,0.475640,11.567152,2.875500,13.354036,0.159860,0.248815,0.064238,0.037139,0.151427,0.0,0.378000,0.269332,0.492110,0.0,0.097853,0.037139,0.471404,0.632290,0.457665,0.097853,0.167825,0.167825,0.187482,19552.576016,51175.460599,0.375589,3.582714,17.935304,0.444445,0.0,0.447727,0.353587,0.497366,0.493092,0.171653
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9999.000000,499999.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.010777,0.185457,0.000000,0.000000,0.000000
25%,98.665198,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,276.000000,161.00000,288.000000,0.000000,0.000000,0.000000,296.250000,0.000000,0.000000,0.000000,70.000000,0.000000,29.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,9999.000000,499999.000000,0.000000,2.000000,0.000000,0.000000,0.0,0.010777,0.185457,0.000000,0.000000,0.000000
50%,130.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,381.000000,242.00000,454.285714,0.000000,0.000000,0.000000,530.000000,10.000000,795.000000,1.000000,74.000000,2.000000,39.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.0,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,9999.00

        12.1.3 - Correlation Matrix

In [182]:
class0_desc = class_0.describe() # saving describe metrics for fisrt class

first = class0_desc[['Purchased Product 1',	'Purchased Product 2',	"Purchased Product 3",	'Purchased Product 4',	'Purchased Product 5',	'Purchased Product 6',	'Purchased Product 7',	'Purchased Product 8']].iloc[1] # selecting interest columns and mean row

first = first.to_frame() # passing to dataframe type

first.rename(columns={'mean': 'Cluster 1'}, inplace=True) # renaming column

first['Houshold Average'] = [classes_frame['Purchased Product 1'].values.mean(), classes_frame['Purchased Product 2'].values.mean(), classes_frame['Purchased Product 3'].values.mean(), classes_frame['Purchased Product 4'].values.mean(), classes_frame['Purchased Product 5'].values.mean(), classes_frame['Purchased Product 6'].values.mean(), classes_frame['Purchased Product 7'].values.mean(), classes_frame['Purchased Product 8'].values.mean()]  # appending columns from other tables

first.reset_index(level=0, inplace=True) # reseting index to numerical

tidy = first.melt(id_vars='index') # melting for bar plot


In [183]:
# plt.figure(figsize=(10,5))

# sns.barplot(x='index', y='value', hue='variable', data=tidy)
# plt.legend(title='Group')
# plt.xticks(rotation='45')
# plt.xlabel('Type of Home Ownership', fontsize=12, weight='bold')
# plt.ylabel('Percentage', fontsize=12, weight='bold')
# #plt.title('Behavioral Comparison between Cluster 1 and all Households', fontsize=15, weight='bold')
# plt.savefig('class1_home.png', dpi=200, bbox_inches='tight')
# plt.show()

In [184]:
class_0_corrmat = abs(class_0.corr()) # absolute correlation

# plt.figure(figsize=(15,10))

# sns.heatmap(class_0_corrmat, square=True, robust=True, cmap='afmhot'); # matrix correlation

    12.2 -  Class 1

        12.2.1 - Glance

In [185]:
class_1.head()

,Length Of Residence,Number of Roadside calss for Year 1,Number of Roadside calss for Year 2,Number of Roadside calss for Year 3,Roadside Member Cost for Year 1,Roadside Member Cost for Year 2,Roadside Member Cost for Year 3,Member Tenure Years,Years to Cancelled Status,Basic Cost,Calculated Tow Miles,Plus Cost,Premier Cost,Total Cost,Total Number of Roadside Calls,Total Roadside Member Cost,Email Available,Age,Years from most recent Membership Expiration,Years from Join Date,Purchased Product 1,Purchased Product 2,Purchased Product 3,Purchased Product 4,Purchased Product 5,Purchased Product 6,Purchased Product 7,Purchased Product 8,Mail Responder,Home Owner,Opt-Out - Publication,Cash Call,Fleet Indicator,Total Number of Purchased Products,Household Purchased Product,Email Correspondent,Home Renter,No Home Ownership Info,City,Number of Children,Annual Income,Education,Annual Credit Possible,Member Status,Renew Method,Motorcycle Indicator,Membership Type,AAA Service Necessity Due Problem 1,AAA Club 1,Vehicle Manufacturer Name 1,AAA Service Necessity Due Problem 2,Vehicle Manufacturer Name 2,AAA Club 2,Number of Members per Household Key,Frequency of use of Roadside Services,Household have used the Roadside Services,Household Group Label,Household Purchase Product Probability,Household Roadside Usage Probability,Membership Type_Basic,Membership Type_Plus,Membership Type_Premier
Household Key,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9690,140.0,0,7,15,0.000000,1470.833333,15059.500000,353,242,3072.500000,22.5,127.5,0.0,4347.500000,22,16530.333333,1,76,0,36,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,1,0,0,WARWICK,0,54500.0,Unknown,774500.0,ACTIVE,AUTO RENEW,0,Basic,Flat Tire,AAA Northeast,SUBARU,,,,6,22,1,1,0.010777,0.999964,1.0,0.0,0.0
87180,150.0,0,0,0,0.000000,0.000000,0.000000,100,2,0.000000,0.0,0.0,0.0,0.000000,0,0.000000,1,68,3,11,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,1,0,0,WARWICK,1,124500.0,Unknown,674500.0,PENDING,NOTICE,0,Basic,Jump Start,AAA Northeast,VOLKSWAGEN,,,,2,0,0,1,0.010777,0.185457,1.0,0.0,0.0
115240,121.0,3,22,10,1961.666667,12205.833333,530.000000,363,93,5885.000000,12.0,0.0,0.0,5885.000000,35,14697.500000,1,50,3,37,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,1,0,0,BARRINGTON,4,124500.0,Unknown,800000.0,ACTIVE,NOTICE,0,Basic,Extrication,AAA Northeast,LEXUS,Flat Tire,,,7,35,1,1,0.010777,1.000000,1.0,0.0,0.0
134220,150.0,2,0,0,1471.250000,0.000000,0.000000,330,242,2161.666667,0.0,0.0,0.0,2161.666667,2,1471.250000,1,70,0,34,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,1,0,0,WAKEFIELD,1,162000.0,Some College,774500.0,ACTIVE,NOTICE,1,Basic,Battery Service,AAA Northeast,ACURA,,,,4,2,1,1,0.010777,0.308831,1.0,0.0,0.0
380930,21.0,0,16,5,0.000000,4437.727273,3084.090909,322,89,997.500000,21.0,0.0,0.0,1085.500000,21,7521.818182,1,46,1,33,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,MANVILLE,0,34500.0,Unknown,724500.0,ACTIVE,NOTICE,0,Basic,Flat Tire,AAA Northeast,BUICK,,,,12,21,1,1,0.010777,0.999967,1.0,0.0,0.0


        12.2.2 - Description

In [186]:
class_1.describe()

,Length Of Residence,Number of Roadside calss for Year 1,Number of Roadside calss for Year 2,Number of Roadside calss for Year 3,Roadside Member Cost for Year 1,Roadside Member Cost for Year 2,Roadside Member Cost for Year 3,Member Tenure Years,Years to Cancelled Status,Basic Cost,Calculated Tow Miles,Plus Cost,Premier Cost,Total Cost,Total Number of Roadside Calls,Total Roadside Member Cost,Email Available,Age,Years from most recent Membership Expiration,Years from Join Date,Purchased Product 1,Purchased Product 2,Purchased Product 3,Purchased Product 4,Purchased Product 5,Purchased Product 6,Purchased Product 7,Purchased Product 8,Mail Responder,Home Owner,Opt-Out - Publication,Cash Call,Fleet Indicator,Total Number of Purchased Products,Household Purchased Product,Email Correspondent,Home Renter,No Home Ownership Info,Number of Children,Annual Income,Annual Credit Possible,Motorcycle Indicator,Number of Members per Household Key,Frequency of use of Roadside Services,Household have used the Roadside Services,Household Group Label,Household Purchase Product Probability,Household Roadside Usage Probability,Membership Type_Basic,Membership Type_Plus,Membership Type_Premier
count,1653.000000,1653.000000,1653.000000,1653.000000,1653.000000,1653.000000,1653.000000,1653.000000,1653.000000,1653.000000,1653.000000,1653.000000,1653.000000,1653.000000,1653.000000,1653.000000,1653.00000,1653.000000,1653.000000,1653.000000,1653.0,1653.0,1653.0,1653.0,1653.0,1653.0,1653.0,1653.0,1653.000000,1653.000000,1653.000000,1653.000000,1653.000000,1653.0,1653.0,1653.000000,1653.000000,1653.000000,1653.000000,1653.000000,1653.000000,1653.000000,1653.000000,1653.000000,1653.000000,1653.0,1.653000e+03,1653.000000,1653.000000,1653.000000,1653.000000
mean,113.705331,3.140351,6.021779,6.020569,824.730443,1451.240783,947.586922,360.215366,240.984271,944.541309,16.372450,24.606323,0.499093,1004.029972,15.182698,3223.558148,0.69631,73.369026,2.297641,37.026618,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.972777,0.957048,0.986086,0.001210,0.660617,0.0,0.0,0.999395,0.039927,0.003025,0.790079,98023.877193,729942.202662,0.178463,4.782819,15.182698,0.720508,1.0,1.077695e-02,0.727850,0.578342,0.398669,0.022989
std,41.178290,6.846087,9.033138,9.079662,2465.472955,3420.188655,2605.065638,134.918537,135.722329,1339.123169,46.299325,147.384585,19.925911,1428.977897,18.232339,5539.016967,0.45999,11.858143,2.709900,13.493986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.162783,0.202811,0.117170,0.034773,0.473643,0.0,0.0,0.024596,0.195848,0.054932,1.007285,57835.018363,69286.760115,0.383018,4.277102,18.232339,0.448885,0.0,5.552795e-17,0.354518,0.493974,0.489773,0.149912
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,25.000000,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,9999.000000,499999.000000,0.000000,1.000000,0.000000,0.000000,1.0,1.077695e-02,0.185457,0.000000,0.000000,0.000000
25%,95.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,264.000000,149.000000,280.000000,0.000000,0.000000,0.000000,284.000000,0.000000,0.000000,0.00000,70.000000,0.000000,27.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.000000,0.000000,54500.000000,674500.000000,0.000000,2.000000,0.000000,0.000000,1.0,1.077695e-02,0.185457,0.000000,0.000000,0.000000
50%,120.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,372.000000,242.000000,467.500000,0.000000,0.000000,0.000000,530.000000,10.000000,750.000000,1.00000,74.000000,1.000000,38.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000,0.000000,1.000000,0.0,0.0,1.000000,0.000000,0.000000,0.000000,94500.000000,774500.000000,0.000000,3.000000,10.000000,1.000000,1.0,1.077695e-02,0.974218,1.000000,0.000000,0.000000
75%,150.000000,3.000000,10.000000,10.000000,200.000000,1105.1

        12.2.3 - Correlation Matrix

In [187]:
classes_frame['Annual Credit Possible'].values.mean()

690444.5001221001

In [188]:
class1_desc = class_1.describe()

first = class1_desc[['Annual Income',	'Annual Credit Possible']].iloc[1]

first = first.to_frame()

first.rename(columns={'mean': 'Cluster 2'}, inplace=True)

first['Houshold Average'] = [classes_frame['Annual Income'].values.mean(), classes_frame['Annual Credit Possible'].values.mean()]

first.reset_index(level=0, inplace=True)

tidy = first.melt(id_vars='index')


In [189]:
# plt.figure(figsize=(10,5))

# sns.barplot(x='index', y='value', hue='variable', data=tidy)
# plt.legend(title='Group')
# plt.xticks(rotation='45')
# plt.xlabel('Income and Credit', fontsize=12, weight='bold')
# plt.ylabel('Values in USD', fontsize=12, weight='bold')
# #plt.title('Behavioral Comparison between Cluster 2 and all Households', fontsize=15, weight='bold')
# plt.savefig('class2_inc.png', dpi=200, bbox_inches='tight')
# plt.show()

In [190]:
class_1_corrmat = abs(class_1.corr())

# plt.figure(figsize=(15,10))

# sns.heatmap(class_1_corrmat, square=True, robust=True, cmap='afmhot'); # matrix correlation

    12.3 - Class 2

        12.3.1 - Glance

In [191]:
class_2.head(n=2)

,Length Of Residence,Number of Roadside calss for Year 1,Number of Roadside calss for Year 2,Number of Roadside calss for Year 3,Roadside Member Cost for Year 1,Roadside Member Cost for Year 2,Roadside Member Cost for Year 3,Member Tenure Years,Years to Cancelled Status,Basic Cost,Calculated Tow Miles,Plus Cost,Premier Cost,Total Cost,Total Number of Roadside Calls,Total Roadside Member Cost,Email Available,Age,Years from most recent Membership Expiration,Years from Join Date,Purchased Product 1,Purchased Product 2,Purchased Product 3,Purchased Product 4,Purchased Product 5,Purchased Product 6,Purchased Product 7,Purchased Product 8,Mail Responder,Home Owner,Opt-Out - Publication,Cash Call,Fleet Indicator,Total Number of Purchased Products,Household Purchased Product,Email Correspondent,Home Renter,No Home Ownership Info,City,Number of Children,Annual Income,Education,Annual Credit Possible,Member Status,Renew Method,Motorcycle Indicator,Membership Type,AAA Service Necessity Due Problem 1,AAA Club 1,Vehicle Manufacturer Name 1,AAA Service Necessity Due Problem 2,Vehicle Manufacturer Name 2,AAA Club 2,Number of Members per Household Key,Frequency of use of Roadside Services,Household have used the Roadside Services,Household Group Label,Household Purchase Product Probability,Household Roadside Usage Probability,Membership Type_Basic,Membership Type_Plus,Membership Type_Premier
Household Key,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1180680,150.0,0,0,0,0.0,0.0,0.0,365,181,530.0,0.0,0.0,0.0,530.0,0,0.0,1,39,2,38,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,1,0,0,CRANSTON,0,14500.0,Unknown,774500.0,CANCELLED,NOTICE,0,Basic,No record for the last three years,No record for the last three years,No record for the last three years,,,,2,0,0,2,0.010777,0.185457,1.0,0.0,0.0
1430510,150.0,0,0,10,0.0,0.0,530.0,490,242,280.0,0.0,0.0,0.0,280.0,10,530.0,0,91,0,50,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,1,0,0,GREENVILLE,2,74500.0,Unknown,674500.0,CANCELLED,NOTICE,0,Basic,No record for the last three years,No record for the last three years,No record for the last three years,,,,1,10,1,2,0.010777,0.974218,1.0,0.0,0.0


        12.3.2 - Describe

In [192]:
class_2.describe()

,Length Of Residence,Number of Roadside calss for Year 1,Number of Roadside calss for Year 2,Number of Roadside calss for Year 3,Roadside Member Cost for Year 1,Roadside Member Cost for Year 2,Roadside Member Cost for Year 3,Member Tenure Years,Years to Cancelled Status,Basic Cost,Calculated Tow Miles,Plus Cost,Premier Cost,Total Cost,Total Number of Roadside Calls,Total Roadside Member Cost,Email Available,Age,Years from most recent Membership Expiration,Years from Join Date,Purchased Product 1,Purchased Product 2,Purchased Product 3,Purchased Product 4,Purchased Product 5,Purchased Product 6,Purchased Product 7,Purchased Product 8,Mail Responder,Home Owner,Opt-Out - Publication,Cash Call,Fleet Indicator,Total Number of Purchased Products,Household Purchased Product,Email Correspondent,Home Renter,No Home Ownership Info,Number of Children,Annual Income,Annual Credit Possible,Motorcycle Indicator,Number of Members per Household Key,Frequency of use of Roadside Services,Household have used the Roadside Services,Household Group Label,Household Purchase Product Probability,Household Roadside Usage Probability,Membership Type_Basic,Membership Type_Plus,Membership Type_Premier
count,688.000000,688.000000,688.000000,688.000000,688.000000,688.000000,688.000000,688.000000,688.000000,688.000000,688.000000,688.000000,688.0,688.000000,688.000000,688.000000,688.000000,688.000000,688.000000,688.000000,688.000000,688.000000,688.000000,688.0,688.000000,688.000000,688.000000,688.000000,688.000000,688.000000,688.000000,688.0,688.0,688.000000,688.000000,688.000000,688.000000,688.000000,688.000000,688.000000,688.000000,688.0,688.000000,688.000000,688.000000,688.0,688.000000,688.000000,688.0,688.0,688.0
mean,113.507326,3.341570,5.997093,6.203488,763.795157,1542.411244,889.334452,359.877907,239.449128,938.131549,19.697467,26.640357,0.0,1075.087477,15.542151,3195.540853,0.699128,72.965116,2.293605,36.973837,0.024709,0.090116,0.004360,0.0,0.043605,0.002907,0.213663,0.114826,0.965116,0.927326,0.986919,1.0,1.0,0.494186,0.372093,0.998547,0.031977,0.040698,0.537791,88534.863372,728606.062500,0.0,4.877907,15.542151,0.716570,2.0,0.370278,0.713669,1.0,0.0,0.0
std,44.481560,6.975809,9.104919,9.443428,2185.087057,3289.155774,2275.049891,129.506058,138.992278,1246.201676,57.479330,183.977569,0.0,2369.605219,18.982976,5094.162507,0.458971,12.690996,2.883647,12.950669,0.155351,0.286557,0.065938,0.0,0.204362,0.053877,0.410190,0.319043,0.183619,0.259790,0.113706,0.0,0.0,0.747608,0.483715,0.038125,0.176066,0.197733,0.828019,56119.892217,75306.274335,0.0,3.972567,18.982976,0.450991,0.0,0.469550,0.356828,0.0,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,23.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9999.000000,499999.000000,0.0,1.000000,0.000000,0.000000,2.0,0.010777,0.185457,1.0,0.0,0.0
25%,90.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,270.750000,144.500000,291.272727,0.000000,0.000000,0.0,300.000000,0.000000,0.000000,0.000000,68.000000,0.000000,28.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.0,1.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,44500.000000,674500.000000,0.0,2.000000,0.000000,0.000000,2.0,0.010777,0.185457,1.0,0.0,0.0
50%,121.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,368.000000,242.000000,481.000000,0.000000,0.000000,0.0,530.000000,10.000000,875.000000,1.000000,74.000000,1.000000,38.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.0,1.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,74500.000000,774500.000000,0.0,4.000000,10.000000,1.000000,2.0,0.010777,0.974218,1.0,0.0,0.0
75%,150.000000,4.000000,10.000000,10.000000,240.113636,1177.000000,

        12.3.3 - Correlation Matrix

In [193]:
class2_desc = class_2.describe()

value = class2_desc['Total Cost'].iloc[1]

first = pd.Series(value, index=[0])

first = first.to_frame()

first.rename(columns={'0': 'Cluster 3'}, inplace=True)

first['Houshold Average'] = [classes_frame['Total Cost'].values.mean()]

first['Cluster 2'] = [class1_desc['Total Cost'].iloc[1]]

first['Cluster 1'] = [class0_desc['Total Cost'].iloc[1]]

first['Cluster 4'] = [class_3.describe()['Total Cost'].iloc[1]]

first.reset_index(level=0, inplace=True)

first.drop(columns='index', inplace=True)

first.rename(columns={0: 'Cluster 3'}, inplace=True)

first = first.T

first.rename(columns={0: 'Average Total Cost'}, inplace=True)

In [194]:
# plt.figure(figsize=(14,5))

# sns.barplot(x=first.index, y=first.values.reshape(-1))
# #plt.legend(title='Group')
# #plt.xticks(rotation='45')
# plt.xlabel('Variables', fontsize=12, weight='bold')
# plt.ylabel('Average Total Cost', fontsize=12, weight='bold')
# plt.title('Average Total Cost Associated to all Clusters and all Households', fontsize=15, weight='bold')
# plt.savefig('class3_cost.png', dpi=200, bbox_inches='tight')
# plt.show()

In [195]:
class_2_corrmat = abs(class_2.corr())

# plt.figure(figsize=(15,10))

# sns.heatmap(class_2_corrmat, square=True, robust=True, cmap='afmhot'); # matrix correlation

    12.4 - Class 3

        12.4.1 - Glance

In [196]:
class_3.head(n=2)

,Length Of Residence,Number of Roadside calss for Year 1,Number of Roadside calss for Year 2,Number of Roadside calss for Year 3,Roadside Member Cost for Year 1,Roadside Member Cost for Year 2,Roadside Member Cost for Year 3,Member Tenure Years,Years to Cancelled Status,Basic Cost,Calculated Tow Miles,Plus Cost,Premier Cost,Total Cost,Total Number of Roadside Calls,Total Roadside Member Cost,Email Available,Age,Years from most recent Membership Expiration,Years from Join Date,Purchased Product 1,Purchased Product 2,Purchased Product 3,Purchased Product 4,Purchased Product 5,Purchased Product 6,Purchased Product 7,Purchased Product 8,Mail Responder,Home Owner,Opt-Out - Publication,Cash Call,Fleet Indicator,Total Number of Purchased Products,Household Purchased Product,Email Correspondent,Home Renter,No Home Ownership Info,City,Number of Children,Annual Income,Education,Annual Credit Possible,Member Status,Renew Method,Motorcycle Indicator,Membership Type,AAA Service Necessity Due Problem 1,AAA Club 1,Vehicle Manufacturer Name 1,AAA Service Necessity Due Problem 2,Vehicle Manufacturer Name 2,AAA Club 2,Number of Members per Household Key,Frequency of use of Roadside Services,Household have used the Roadside Services,Household Group Label,Household Purchase Product Probability,Household Roadside Usage Probability,Membership Type_Basic,Membership Type_Plus,Membership Type_Premier
Household Key,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8750,90.558333,8,4,7,4623.928571,2522.142857,5472.0,289,131,1250.357143,80.142857,265.714286,0.0,1516.071429,19,12618.071429,1,74,2,30,0,1,0,0,0,0,0,0,1,1,1,0,0,1,1,1,0,0,WEST WARWICK,1,34500.0,Some College,624500.0,ACTIVE,AUTO RENEW,0,Basic,Battery Service,AAA Northeast,TOYOTA,,,,18,19,1,3,0.990008,0.999183,1.0,0.0,0.0
33380,50.000000,0,0,0,0.000000,0.000000,0.0,373,260,0.000000,0.000000,0.000000,0.0,0.000000,0,0.000000,0,90,11,38,0,0,0,0,0,0,1,0,1,1,1,0,0,1,1,1,0,0,CENTRAL FALLS,0,54500.0,Some College,674500.0,ACTIVE,NOTICE,0,Basic,Lockout,AAA Northeast,INFINITI,,,,3,0,0,3,0.997030,0.185457,1.0,0.0,0.0


        12.4.2 - Describe

In [197]:
class_3.describe()

,Length Of Residence,Number of Roadside calss for Year 1,Number of Roadside calss for Year 2,Number of Roadside calss for Year 3,Roadside Member Cost for Year 1,Roadside Member Cost for Year 2,Roadside Member Cost for Year 3,Member Tenure Years,Years to Cancelled Status,Basic Cost,Calculated Tow Miles,Plus Cost,Premier Cost,Total Cost,Total Number of Roadside Calls,Total Roadside Member Cost,Email Available,Age,Years from most recent Membership Expiration,Years from Join Date,Purchased Product 1,Purchased Product 2,Purchased Product 3,Purchased Product 4,Purchased Product 5,Purchased Product 6,Purchased Product 7,Purchased Product 8,Mail Responder,Home Owner,Opt-Out - Publication,Cash Call,Fleet Indicator,Total Number of Purchased Products,Household Purchased Product,Email Correspondent,Home Renter,No Home Ownership Info,Number of Children,Annual Income,Annual Credit Possible,Motorcycle Indicator,Number of Members per Household Key,Frequency of use of Roadside Services,Household have used the Roadside Services,Household Group Label,Household Purchase Product Probability,Household Roadside Usage Probability,Membership Type_Basic,Membership Type_Plus,Membership Type_Premier
count,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.0,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.0,1029.0,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.0,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000
mean,115.644522,2.810496,5.812439,6.135083,790.635860,1540.075986,972.915303,362.094266,242.940719,909.461428,16.706126,24.035438,0.0,967.107867,14.758017,3303.627148,0.715258,73.894072,2.284742,37.201166,0.112731,0.223518,0.011662,0.000972,0.087464,0.002915,0.551020,0.316812,0.978620,0.952381,0.996113,0.001944,0.672498,1.307094,1.0,1.0,0.039845,0.007775,0.729835,92921.268222,727104.932945,0.188533,4.761905,14.758017,0.709427,3.0,0.982852,0.713426,0.531584,0.432459,0.035957
std,44.884149,6.324713,9.081602,9.111620,3028.008985,3968.813104,2714.766129,133.953212,129.326922,1259.175630,39.766510,92.991974,0.0,1324.353456,17.456526,6341.200301,0.451511,11.824818,2.815944,13.404053,0.316417,0.416805,0.107411,0.031174,0.282651,0.053942,0.497632,0.465460,0.144718,0.213062,0.062257,0.044065,0.469530,0.565684,0.0,0.0,0.195689,0.087873,1.001407,55249.756690,68913.139244,0.391327,9.865667,17.456526,0.454248,0.0,0.062258,0.358682,0.499244,0.495658,0.186274
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,26.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.0,1.0,0.000000,0.000000,0.000000,9999.000000,499999.000000,0.000000,1.000000,0.000000,0.000000,3.0,0.688947,0.185457,0.000000,0.000000,0.000000
25%,95.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,270.000000,166.000000,290.000000,0.000000,0.000000,0.0,300.000000,0.000000,0.000000,0.000000,70.000000,0.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.0,1.0,0.000000,0.000000,0.000000,54500.000000,674500.000000,0.000000,2.000000,0.000000,0.000000,3.0,0.994138,0.185457,0.000000,0.000000,0.000000
50%,120.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,375.000000,242.000000,456.666667,0.000000,0.000000,0.0,503.333333,10.000000,655.000000,1.000000,74.000000,1.000000,38.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.0,1.0,0.000000

        12.4.3 - Correlation Matrix

In [198]:
classes_frame['Total Number of Purchased Products'].values.mean()

0.47741147741147744

In [199]:
class3_desc = class_3.describe()

value = class3_desc['Total Number of Purchased Products'].iloc[1]

first = pd.Series(value, index=[0])

first = first.to_frame()

first.rename(columns={'0': 'Cluster 4'}, inplace=True)

first['Houshold Average'] = [classes_frame['Total Number of Purchased Products'].values.mean()]

first['Cluster 2'] = [class1_desc['Total Number of Purchased Products'].iloc[1]]

first['Cluster 1'] = [class0_desc['Total Number of Purchased Products'].iloc[1]]

first['Cluster 3'] = [class2_desc['Total Number of Purchased Products'].iloc[1]]

first.reset_index(level=0, inplace=True)

first.drop(columns='index', inplace=True)

first.rename(columns={0: 'Cluster 4'}, inplace=True)

first = first.T

first.rename(columns={0: 'Average Total Number of Purchased Products'}, inplace=True)

In [200]:
# plt.figure(figsize=(14,5))

# sns.barplot(x=first.index, y=first.values.reshape(-1))
# #plt.legend(title='Group')
# #plt.xticks(rotation='45')
# plt.xlabel('Variables', fontsize=12, weight='bold')
# plt.ylabel('Average Total Number of Purchased Products', fontsize=12, weight='bold')
# plt.title('Average Total Number of Purchased Products per Cluster and all Households', fontsize=15, weight='bold')
# plt.savefig('class3_cost.png', dpi=200, bbox_inches='tight')
# plt.show()

In [201]:
class_3_corrmat = abs(class_3.corr())

# plt.figure(figsize=(15,10))

# sns.heatmap(class_3_corrmat, square=True, robust=True, cmap='afmhot'); # matrix correlation